<a href="https://colab.research.google.com/github/IamImtiazChy/thesis-research-2025/blob/main/Assertion_Accuracy_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Framework for Maximizing Assertion Accuracy in LLM-Generated Test Code
## 1. Introduction and Setup

This notebook implements a framework for maximizing assertion accuracy in LLM-generated test code. It covers data collection from GitHub, HumanEval+, and MBPP, test generation using DeepSeek and Microsoft Phi LLMs, mutation testing for assertion evaluation, and an assertion correction mechanism.

**Research Objectives:**
1.  Collect and preprocess code and test data from diverse sources.
2.  Utilize DeepSeek and Phi LLMs (downloadable models) to generate test cases for Python functions.
3.  Evaluate the quality of LLM-generated assertions using mutation testing.
4.  Develop and apply a model/technique to correct or refine weak/incorrect assertions.
5.  Analyze the overall effectiveness of the framework.


### 1.1 Install Necessary Libraries


In [ ]:
!pip install -q PyGithub pandas datasets transformers accelerate bitsandbytes pyarrow torch


### 1.2 Import Required Modules


In [ ]:
import pandas as pd
import re
from github import Github
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import ast
import importlib.util
import sys
import traceback
import random
import time
from IPython.display import display, Markdown


### 1.3 Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define base path for project data on Google Drive
BASE_DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Assertion-Accuracy-Research/Data"
os.makedirs(BASE_DRIVE_PATH, exist_ok=True)
print(f"Data will be stored in: {BASE_DRIVE_PATH}")


Mounted at /content/drive
Data will be stored in: /content/drive/MyDrive/Colab Notebooks/Assertion-Accuracy-Research/Data


### 1.4 GitHub Authentication (using Colab Secrets)


In [ ]:
from google.colab import userdata
g = None
try:
    ACCESS_TOKEN = userdata.get('GITHUB_TOKEN')
    if not ACCESS_TOKEN:
        print("GitHub Access Token not found in Colab Secrets. Please add it as 'GITHUB_TOKEN'.")
        print("GitHub data collection will be skipped if token is not available.")
    else:
        g = Github(ACCESS_TOKEN)
        github_user = g.get_user()
        print(f"Successfully authenticated to GitHub as: {github_user.login}")
except Exception as e:
    print(f"Could not retrieve GITHUB_TOKEN from Colab Secrets or authentication failed: {e}")
    print("GitHub data collection will be skipped.")


Successfully authenticated to GitHub as: IamImtiazChy


## 2. Data Collection and Preprocessing

This section covers collecting data from GitHub repositories, HumanEval+, and MBPP datasets.


### 2.1 GitHub Data Collection
This cell contains the functions to extract data from GitHub. It will only run if the GitHub token was successfully loaded.


In [ ]:
# Install PyGithub and Import Libraries
"""
!pip install PyGithub pandas
"""

import pandas as pd
import re
from github import Github
import os

In [ ]:
# GitHub Authentication and Helper Functions

from google.colab import userdata
ACCESS_TOKEN = userdata.get("GITHUB_TOKEN")

if not ACCESS_TOKEN:
    print("GitHub Access Token not found. Please set it up in Colab Secrets as GITHUB_TOKEN.")
    # Or raise an error / stop execution
    g = None
else:
    try:
        g = Github(ACCESS_TOKEN)
        user = g.get_user()
        print(f"Successfully authenticated to GitHub as: {user.login}")
    except Exception as e:
        print(f"GitHub authentication failed: {e}")
        g = None

def extract_functions_tests_errors(code):
    """Extract functions, test cases, and error messages (assertions) from code."""
    # Adjusted patterns for better matching and to avoid overly greedy matches
    # Function pattern: def function_name(params): ... (multiline, non-greedy body)
    function_pattern = r"def\s+(?!test_)\w+\(.*?\):(?:\s*\"\"\"[^\"]*\"\"\"|\s*\'\'\'.*?\'\'\')?\s*\n(?:^\s{4,}.*\n|\s*\n)*"
    # Test pattern: def test_function_name(params): ... (multiline, non-greedy body)
    test_pattern = r"def\s+test_\w+\(.*?\):(?:\s*\"\"\"[^\"]*\"\"\"|\s*\'\'\'.*?\'\'\')?\s*\n(?:^\s{4,}.*\n|\s*\n)*"
    # Assertion pattern: assert ... (single line or simple multiline)
    # This pattern is simplified; robust assertion extraction can be complex.
    assertion_pattern = r"^\s*assert .*?(?=\n\s*[^\s#]|$)" # Matches assert until end of line or next non-indented line

    functions = re.findall(function_pattern, code, re.MULTILINE)
    test_cases = re.findall(test_pattern, code, re.MULTILINE)
    assertions = re.findall(assertion_pattern, code, re.MULTILINE)

    return {
        "functions": functions,
        "test_cases": test_cases,
        "assertions": assertions, # Renamed from error_messages for clarity
    }

def process_repository(repo_name):
    """Process a single repository and extract code components."""
    if not g:
        print(f"GitHub client not initialized. Skipping repository {repo_name}.")
        return []
    try:
        print(f"Fetching repository: {repo_name}")
        repo = g.get_repo(repo_name)
        repo_data = []
        contents_to_process = []

        # Initial contents from the root directory
        try:
            root_contents = repo.get_contents("")
            contents_to_process.extend(root_contents)
        except Exception as e:
            print(f"Could not get root contents for {repo_name}: {e}")
            return []

        processed_paths = set()
        file_count = 0
        max_files_to_process = 200 # Limit to avoid very long processing times in Colab for large repos

        while contents_to_process and file_count < max_files_to_process:
            file_content = contents_to_process.pop(0)

            if file_content.path in processed_paths:
                continue
            processed_paths.add(file_content.path)

            if file_content.type == "dir":
                try:
                    print(f"  Exploring directory: {file_content.path}")
                    dir_contents = repo.get_contents(file_content.path)
                    contents_to_process.extend(dir_contents)
                except Exception as e:
                    print(f"  Warning: Could not get contents of dir {file_content.path}: {e}")
            elif file_content.name.endswith(".py"):
                file_count += 1
                print(f"  Processing file ({file_count}): {file_content.path}")
                try:
                    # Check if file is too large to avoid memory issues
                    if file_content.size > 1000000: # 1MB limit for individual files
                        print(f"    Skipping large file: {file_content.path} (size: {file_content.size} bytes)")
                        continue
                    code = file_content.decoded_content.decode("utf-8", errors="ignore")
                    extracted = extract_functions_tests_errors(code)

                    if any(extracted.values()): # If any list (functions, tests, assertions) is not empty
                        data = {
                            "repo_name": repo_name,
                            "file_path": file_content.path,
                            "functions_count": len(extracted["functions"]),
                            "test_cases_count": len(extracted["test_cases"]),
                            "assertions_count": len(extracted["assertions"]),
                            "raw_functions": extracted["functions"],
                            "raw_test_cases": extracted["test_cases"],
                            "raw_assertions": extracted["assertions"],
                        }
                        repo_data.append(data)
                except Exception as e:
                    print(f"    Error processing file {file_content.path}: {str(e)}")

        if file_count >= max_files_to_process:
            print(f"Reached max file processing limit ({max_files_to_process}) for repository {repo_name}.")
        return repo_data

    except Exception as e:
        print(f"Error processing repository {repo_name}: {str(e)}")
        return []

def create_github_dataset(repositories, drive_output_path):
    """Create structured dataset from list of repositories and save to Google Drive."""
    all_data = []
    total_repos = len(repositories)

    for idx, repo_name in enumerate(repositories, 1):
        print(f"\nProcessing repository [{idx}/{total_repos}]: {repo_name}")
        print("=" * 50)

        repo_data = process_repository(repo_name)
        all_data.extend(repo_data)

        print(f"Completed {repo_name}: Found {len(repo_data)} Python files with relevant code snippets.")
        print(f"Progress: {idx/total_repos*100:.1f}%")

    if not all_data:
        print("\nNo data collected from GitHub repositories.")
        return None

    print("\nCreating final GitHub dataset...")
    df = pd.DataFrame(all_data)

    try:
        os.makedirs(os.path.dirname(drive_output_path), exist_ok=True)
        df.to_csv(drive_output_path, index=False)
        print(f"\nGitHub dataset saved successfully to {drive_output_path}")
        print(f"Total Python files processed and included: {len(df)}")
    except Exception as e:
        print(f"\nError saving GitHub dataset to {drive_output_path}: {e}")
        print("Displaying DataFrame head as fallback:")
        print(df.head())

    return df

Successfully authenticated to GitHub as: IamImtiazChy


In [ ]:
# Run GitHub Data Collection
print("Running GitHub Data Collection for Colab...")
# For the full research, expand this list or use a more systematic way to select repos.
repositories_to_process = ["pallets/flask", "requests/requests"]

# Define output path in Google Drive
github_csv_path = "/content/drive/MyDrive/Colab Notebooks/Assertion-Accuracy-Research/Data/github_dataset.csv"

github_df = create_github_dataset(repositories_to_process, github_csv_path)

if github_df is not None:
    print("\nGitHub Dataset Statistics:")
    print(f"Total number of files processed: {len(github_df)}")
    print(f"Total functions extracted: {github_df['functions_count'].sum()}")
    print(f"Total test cases extracted: {github_df['test_cases_count'].sum()}")
    print(f"Total assertions found: {github_df['assertions_count'].sum()}")
    print("\nGitHub Dataset Preview (first 5 rows):")
    from IPython.display import display # For Colab
    display(github_df.head())
elif g is None:
    print("Skipping GitHub data collection as GitHub client is not initialized.")
else:
    print("This script is intended to be run in a Google Colab environment or adapted for local use.")

### 2.2 HumanEval+ Dataset Collection


In [ ]:
# Install/Ensure pandas and pyarrow are available
"""
!pip install pandas pyarrow
"""

import pandas as pd
import os
import re

In [ ]:
# Helper function to extract assertions
def extract_assertions_from_test_code(test_code_str):
    """Extracts assertion statements from a string of test code using regex."""
    # This is a simplified regex. AST parsing would be more robust but adds complexity.
    assertion_pattern = r"^\s*assert .*?(?=\n\s*[^\s#]|$)" # Matches assert until end of line or next non-indented line
    assertions = re.findall(assertion_pattern, test_code_str, re.MULTILINE)
    return assertions


In [ ]:
# Load and Process HumanEval+ Dataset
def load_and_process_humanevalplus(drive_output_path):
    """Loads HumanEval+ dataset, processes it, and saves to Google Drive."""
    print("Loading HumanEval+ dataset from Hugging Face Hub...")
    try:
        parquet_url = "hf://datasets/evalplus/humanevalplus/data/test-00000-of-00001-5973903632b82d40.parquet"
        df = pd.read_parquet(parquet_url)
        print(f"Successfully loaded HumanEval+ dataset. Shape: {df.shape}")
    except Exception as e:
        print(f"Error loading HumanEval+ dataset: {e}")
        return None

    processed_data = []
    print("Processing HumanEval+ data...")
    for index, row in df.iterrows():
        try:
            task_id = row.get("task_id", f"humaneval_{index}")
            # Combine prompt and canonical_solution to form the full function code
            function_code = row.get("prompt", "") + row.get("canonical_solution", "")
            test_code = row.get("test", "")
            entry_point = row.get("entry_point", "")

            if not function_code.strip() or not test_code.strip():
                print(f"Skipping entry {task_id} due to missing function or test code.")
                continue

            # Ensure function_code is a complete, parseable function definition
            # The prompt often is `def func_name():\n` and solution is `  return ...`
            # We might need to ensure proper indentation if solution doesn't start with it.
            # For simplicity, we assume they combine correctly here.

            assertions = extract_assertions_from_test_code(test_code)

            processed_data.append({
                "dataset_source": "HumanEval+",
                "function_id": task_id,
                "function_code": function_code.strip(),
                "test_code": test_code.strip(),
                "assertions": assertions,
                "assertions_count": len(assertions),
                "entry_point": entry_point
            })
        except Exception as e:
            print(f"Error processing row {index} from HumanEval+: {e}")
            continue

    if not processed_data:
        print("No data processed from HumanEval+.")
        return None

    processed_df = pd.DataFrame(processed_data)
    print(f"Successfully processed {len(processed_df)} entries from HumanEval+.")

    try:
        os.makedirs(os.path.dirname(drive_output_path), exist_ok=True)
        processed_df.to_csv(drive_output_path, index=False)
        print(f"Processed HumanEval+ dataset saved to {drive_output_path}")
    except Exception as e:
        print(f"Error saving processed HumanEval+ dataset to {drive_output_path}: {e}")
        print("Displaying DataFrame head as fallback:")
        print(processed_df.head())

    return processed_df


In [ ]:
# Run HumanEval+ Data Collection and Processing
print("Running HumanEval+ Data Collection and Processing for Colab...")

humaneval_csv_path = "/content/drive/MyDrive/Colab Notebooks/Assertion-Accuracy-Research/Data/humanevalplus_dataset.csv"

humaneval_df = load_and_process_humanevalplus(humaneval_csv_path)

if humaneval_df is not None:
    print("\nHumanEval+ Processed Dataset Statistics:")
    print(f"Total number of entries: {len(humaneval_df)}")
    print(f"Total assertions extracted: {humaneval_df['assertions_count'].sum()}")
    print("\nHumanEval+ Processed Dataset Preview (first 5 rows):")
    from IPython.display import display # For Colab
    display(humaneval_df.head())
else:
    print("This script is intended to be run in a Google Colab environment or adapted for local use.")

### 2.3 MBPP Dataset Collection


In [ ]:
# Install/Ensure datasets, pandas are available
"""
!pip install -q datasets pandas
"""

import pandas as pd
import ast
import os
import json
import re

In [ ]:
# Load and Process MBPP Dataset

# Helper function to extract assertions from test code string
def extract_assertions_from_test(test_code_str):
    assertions = []
    if not isinstance(test_code_str, str):
        return assertions # Return empty if not a string
    try:
        tree = ast.parse(test_code_str)
        for node in ast.walk(tree):
            if isinstance(node, ast.Assert):
                try:
                    # ast.unparse requires Python 3.9+. Colab usually has a recent version.
                    assertion_text = ast.unparse(node.test).strip()
                except AttributeError:
                    # Basic fallback for older ast or if unparse is not available/fails
                    assertion_text = f"Assertion involving {type(node.test.ops[0]).__name__ if hasattr(node.test, 'ops') and node.test.ops else 'comparison'}"
                assertions.append(assertion_text)
    except SyntaxError:
        # print(f"SyntaxError while parsing test code for assertions: {test_code_str[:100]}...")
        pass # Or log an error
    return assertions

def load_and_process_mbpp(output_csv_path):
    """
    Loads the MBPP dataset directly from Hugging Face Parquet files using pandas,
    processes it to extract relevant information, and saves it to a CSV file.
    This method is more robust against Colab caching issues with the datasets library.
    """
    print("Loading MBPP dataset directly from Hugging Face Parquet files (google-research-datasets/mbpp)...")

    # Using the 'test' split as per previous requirements and user example
    # The user provided: df = pd.read_parquet("hf://datasets/google-research-datasets/mbpp/" + splits["train"])
    # We need the 'test' split: 'full/test-00000-of-00001.parquet'
    # The dataset name on HF Hub is "google/mbpp" or "google-research-datasets/mbpp"
    # The user's example used "google-research-datasets/mbpp"
    # The structure from user: 'task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'

    mbpp_parquet_url = "hf://datasets/google-research-datasets/mbpp/full/test-00000-of-00001.parquet"
    # Alternative if the above doesn't work, sometimes it's just 'mbpp' as dataset name
    # mbpp_parquet_url_alt = "hf://datasets/mbpp/full/test-00000-of-00001.parquet"

    try:
        print(f"Attempting to load MBPP test split from: {mbpp_parquet_url}")
        mbpp_df = pd.read_parquet(mbpp_parquet_url)
        print("MBPP dataset (test split) loaded successfully via pandas.read_parquet.")
    except Exception as e:
        print(f"Error loading MBPP dataset from {mbpp_parquet_url}: {e}")
        # print(f"Attempting to load MBPP test split from alternative URL: {mbpp_parquet_url_alt}")
        # try:
        #     mbpp_df = pd.read_parquet(mbpp_parquet_url_alt)
        #     print("MBPP dataset (test split) loaded successfully via pandas.read_parquet from alternative URL.")
        # except Exception as e2:
        #     print(f"Error loading MBPP dataset from alternative URL {mbpp_parquet_url_alt}: {e2}")
        print("Please ensure have `pyarrow` and `fsspec` with `hf-transfer` installed if loading directly from hf:// URIs.")
        print("In Colab, might need: !pip install -q pyarrow fsspec hf-transfer")
        return None

    data_list = []
    for index, item in mbpp_df.iterrows():
        function_id = item.get("task_id")
        # The 'text' column contains the prompt/description
        prompt_text = item.get("text")
        # The 'code' column contains the function code
        function_code = item.get("code")
        # 'test_list' is a list of assertion strings
        test_code_list = item.get("test_list", [])
        # 'test_setup_code' is any setup code needed for tests
        test_setup_code = item.get("test_setup_code", "")
        entry_point = None

        if isinstance(function_code, str) and function_code.strip():
            try:
                parsed_code = ast.parse(function_code)
                for node in parsed_code.body:
                    if isinstance(node, ast.FunctionDef):
                        entry_point = node.name
                        break
            except SyntaxError:
                # print(f"SyntaxError parsing function code for entry point: {function_id}")
                pass

        # Combine test_setup_code and test_list to form the full test code string
        full_test_code_str = str(test_setup_code) if pd.notna(test_setup_code) else ""
        if isinstance(test_code_list, list) and test_code_list:
            full_test_code_str += ("\n" if full_test_code_str else "") + "\n".join(test_code_list)

        assertions = extract_assertions_from_test(full_test_code_str)

        data_list.append({
            "dataset_source": "mbpp",
            "function_id": str(function_id), # Ensure ID is string for consistency
            "prompt": str(prompt_text) if pd.notna(prompt_text) else "",
            "function_code": str(function_code) if pd.notna(function_code) else "",
            "test_code": full_test_code_str,
            "assertions": json.dumps(assertions), # Store assertions as a JSON string
            "assertions_count": len(assertions),
            "entry_point": entry_point
        })

    if not data_list:
        print("No data extracted from MBPP dataset after processing.")
        return None

    processed_df = pd.DataFrame(data_list)

    # Ensure output directory exists
    output_dir = os.path.dirname(output_csv_path)
    if output_dir and not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir, exist_ok=True)
            print(f"Created directory: {output_dir}")
        except OSError as e:
            print(f"Error creating directory {output_dir}: {e}. Attempting to save to current directory.")
            # Fallback to saving in the current directory if dir creation fails
            output_csv_path = os.path.basename(output_csv_path)

    try:
        processed_df.to_csv(output_csv_path, index=False)
        print(f"MBPP dataset processed and saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving MBPP processed data to CSV at {output_csv_path}: {e}")
        return None

    return processed_df


In [ ]:
# Run MBPP Data Collection and Processing
print("Running MBPP Data Collection and Processing for Colab...")

mbpp_csv_path = "/content/drive/MyDrive/Colab Notebooks/Assertion-Accuracy-Research/Data/mbpp_dataset.csv"

mbpp_df = load_and_process_mbpp(mbpp_csv_path)

if mbpp_df is not None:
    print("\nMBPP Processed Dataset Statistics:")
    print(f"Total number of entries: {len(mbpp_df)}")
    print(f"Total assertions extracted: {mbpp_df['assertions_count'].sum()}")
    print("\nMBPP Processed Dataset Preview (first 5 rows):")
    from IPython.display import display # For Colab
    display(mbpp_df.head())
else:
    print("Failed to load and process the MBPP dataset.")

### 2.4 Combine and Finalize Datasets


In [ ]:
from IPython.display import display # For Colab


print("\n--- Combining Datasets ---")

# Adding a check here for robustness in case this cell is run out of order or BASE_DRIVE_PATH is not set.
if "BASE_DRIVE_PATH" not in locals() and "BASE_DRIVE_PATH" not in globals():
    print("Error: BASE_DRIVE_PATH is not defined. Please ensure it's set in a previous cell.")
    # Attempt to define a default if running in Colab, otherwise this will fail if not in Colab
    try:
        if "google.colab" in str(get_ipython()):
            BASE_DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Assertion-Accuracy-Research/Data"
            print(f"Attempting to use default BASE_DRIVE_PATH: {BASE_DRIVE_PATH}")
            os.makedirs(BASE_DRIVE_PATH, exist_ok=True) # Ensure it exists
        else:
            raise NameError("BASE_DRIVE_PATH not defined and not in Colab to set default.")
    except NameError:
        print("Cannot determine BASE_DRIVE_PATH. Please define it manually.")
        # Exit or raise error if BASE_DRIVE_PATH is critical and not found
        raise ValueError("BASE_DRIVE_PATH must be defined to proceed with combining datasets.")

github_csv_path = os.path.join(BASE_DRIVE_PATH, "github_dataset.csv")
humaneval_csv_path = os.path.join(BASE_DRIVE_PATH, "humanevalplus_dataset.csv")
mbpp_csv_path = os.path.join(BASE_DRIVE_PATH, "mbpp_dataset.csv")
combined_dataset_path = os.path.join(BASE_DRIVE_PATH, "combined_dataset.csv")

all_dfs = []

print("Note: GitHub data combination requires careful schema alignment and transformation (e.g., exploding lists of functions/tests per file into one function per row). This simplified version will focus on HumanEval+ and MBPP for the combined dataset if GitHub data is not pre-transformed.")
df_github_transformed = None # Placeholder for transformed GitHub data

if os.path.exists(github_csv_path):
    try:
        df_github_raw = pd.read_csv(github_csv_path)
        print(f"Loaded raw GitHub data with {len(df_github_raw)} file entries. This data needs transformation to be combined effectively.")
        # Placeholder for transformation logic. For the purpose of this script, we will not combine it directly
        # unless it's already in a compatible one-function-per-row format with necessary columns.
        # Example columns needed: 'dataset_source', 'function_id', 'function_code', 'test_code', 'assertions', 'entry_point'
    except Exception as e:
        print(f"Error loading or initially processing GitHub CSV: {e}")
else:
    print(f"GitHub dataset not found at {github_csv_path}. Skipping its inclusion in combined dataset for now.")

if os.path.exists(humaneval_csv_path):
    try:
        df_humaneval = pd.read_csv(humaneval_csv_path)
        # Ensure required columns exist
        required_cols_he = ['dataset_source', 'function_id', 'function_code', 'test_code', 'assertions', 'entry_point']
        if all(col in df_humaneval.columns for col in required_cols_he):
            df_humaneval = df_humaneval[required_cols_he].copy()
            all_dfs.append(df_humaneval)
            print(f"Loaded HumanEval+ data: {len(df_humaneval)} entries.")
        else:
            print(f"HumanEval+ CSV at {humaneval_csv_path} is missing one or more required columns: {required_cols_he}")
    except Exception as e:
        print(f"Error loading HumanEval+ CSV from {humaneval_csv_path}: {e}")
else:
    print(f"HumanEval+ dataset not found at {humaneval_csv_path}. Skipping.")

if os.path.exists(mbpp_csv_path):
    try:
        df_mbpp = pd.read_csv(mbpp_csv_path)
        required_cols_mbpp = ['dataset_source', 'function_id', 'function_code', 'test_code', 'assertions', 'entry_point']
        if all(col in df_mbpp.columns for col in required_cols_mbpp):
            df_mbpp = df_mbpp[required_cols_mbpp].copy()
            all_dfs.append(df_mbpp)
            print(f"Loaded MBPP data: {len(df_mbpp)} entries.")
        else:
            print(f"MBPP CSV at {mbpp_csv_path} is missing one or more required columns: {required_cols_mbpp}")
    except Exception as e:
        print(f"Error loading MBPP CSV from {mbpp_csv_path}: {e}")
else:
    print(f"MBPP dataset not found at {mbpp_csv_path}. Skipping.")

if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.dropna(subset=['function_code'], inplace=True)
    combined_df['function_code'] = combined_df['function_code'].astype(str)
    combined_df['function_id'] = combined_df['function_id'].astype(str)
    combined_df['dataset_source'] = combined_df['dataset_source'].astype(str)

    combined_df['unique_id'] = combined_df['dataset_source'] + '_' + combined_df['function_id']

    combined_df.drop_duplicates(subset=['function_code'], keep='first', inplace=True)
    combined_df.drop_duplicates(subset=['unique_id'], keep='first', inplace=True) # Also ensure unique_id is unique

    combined_df.to_csv(combined_dataset_path, index=False)
    print(f"\nCombined dataset (from available sources) saved to {combined_dataset_path} with {len(combined_df)} unique functions.")
    print("\nCombined Dataset Preview:")
    display(combined_df.head())
    print("\nCombined Dataset Info:")
    combined_df.info()
else:
    print("No datasets were loaded to combine. Ensure HumanEval+ and/or MBPP CSVs exist, are readable, and contain required columns.")
    # Create an empty DataFrame with expected columns to prevent downstream errors if combined_df is used
    expected_cols = ['dataset_source', 'function_id', 'function_code', 'test_code', 'assertions', 'entry_point', 'unique_id']
    combined_df = pd.DataFrame(columns=expected_cols)
    # Save the empty df so the path exists
    combined_df.to_csv(combined_dataset_path, index=False)
    print(f"Empty combined dataset template saved to {combined_dataset_path}")


## 3. LLM Setup and Test Generation

This section focuses on setting up the DeepSeek and Microsoft Phi LLMs (downloadable models) and using them to generate test cases for the functions in our combined dataset.


In [ ]:
# It handles LLM setup, test generation, and baseline evaluation.

import pandas as pd
import os
import json
import ast
import time
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from IPython.display import display, Markdown # For Colab

# --- LLM Loading Functions ---
def load_llm_model(model_name, model_id, task="text-generation", use_quantization=False):
    """Loads a specified LLM from Hugging Face with optional quantization."""
    print(f"Loading LLM: {model_name} ({model_id})...")
    quantization_config = None
    if use_quantization and torch.cuda.is_available():
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )
        print("Applying 4-bit quantization.")

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        model_args = {"trust_remote_code": True}
        if quantization_config:
            model_args["quantization_config"] = quantization_config
            model_args["device_map"] = "auto" # device_map auto is good for quantized models
        else:
            # For non-quantized, device_map="auto" might put it on CPU if VRAM is low.
            # Explicitly try to use CUDA if available and not quantizing, otherwise auto.
            model_args["device_map"] = "cuda" if torch.cuda.is_available() else "auto"

        model = AutoModelForCausalLM.from_pretrained(model_id, **model_args)

        if tokenizer.pad_token_id is None:
            tokenizer.pad_token_id = tokenizer.eos_token_id
            model.config.pad_token_id = model.config.eos_token_id

        # Determine torch_dtype based on availability and quantization
        dtype_to_use = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16
        if quantization_config: # Quantized models often handle dtype internally or prefer bfloat16 for compute
             dtype_to_use = torch.bfloat16 # As specified in bnb_4bit_compute_dtype

        pipe = pipeline(task, model=model, tokenizer=tokenizer, torch_dtype=dtype_to_use if not quantization_config else None)
        print(f"{model_name} loaded successfully on device: {model.device}")
        return pipe
    except Exception as e:
        print(f"Error loading {model_name} ({model_id}): {e}")
        print("Ensure have accepted any necessary terms on Hugging Face Hub for this model and have internet access.")
        print("If using quantization, ensure CUDA is available and drivers are up to date.")
        return None

# --- Test Generation Function ---
def generate_tests_with_llm(llm_pipeline, function_code, function_name, prompt_template, max_new_tokens=512, num_return_sequences=1):
    """Generates test code for a given function using the loaded LLM pipeline."""
    full_prompt = prompt_template.format(function_code=function_code, function_name=function_name)
    try:
        model_identifier = llm_pipeline.model.config._name_or_path if hasattr(llm_pipeline.model, "config") and hasattr(llm_pipeline.model.config, "_name_or_path") else "Unknown LLM"
        print(f"Generating tests for {function_name} using {model_identifier}...")
        sequences = llm_pipeline(
            full_prompt,
            max_new_tokens=max_new_tokens,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            pad_token_id=llm_pipeline.tokenizer.eos_token_id
        )

        generated_tests = []
        for seq in sequences:
            generated_code = seq["generated_text"]
            code_start_marker = "python\n"
            code_end_marker = "\n"

            # Try to find the last occurrence of the code block
            start_index = generated_code.rfind(code_start_marker)
            if start_index != -1:
                start_index += len(code_start_marker)
                end_index = generated_code.find(code_end_marker, start_index)
                if end_index != -1:
                    extracted_code = generated_code[start_index:end_index]
                else: # If no end marker after the last start marker, take rest of string
                    extracted_code = generated_code[start_index:]
                generated_tests.append(extracted_code.strip())
            else:
                # Fallback: if no python marker, try to get text after the prompt
                prompt_end_index = len(full_prompt)
                if len(generated_code) > prompt_end_index:
                     extracted_code = generated_code[prompt_end_index:].strip()
                     if not extracted_code.startswith(("def test_", "import", "assert")):
                         print(f"Warning: Generated test for {function_name} (fallback extraction) might not be well-formed code. Raw output: {extracted_code[:200]}...")
                     generated_tests.append(extracted_code)
                else:
                    generated_tests.append(generated_code.strip())

        print(f"Successfully generated {len(generated_tests)} test sequence(s) for {function_name}.")
        return generated_tests[0] if generated_tests and num_return_sequences == 1 else generated_tests

    except Exception as e:
        print(f"Error generating tests for {function_name} with {model_identifier}: {e}")
        return None

# --- Assertion Extraction and Basic Evaluation ---
def extract_assertions_from_code(code_string):
    assertions = []
    if not isinstance(code_string, str):
        return assertions
    try:
        tree = ast.parse(code_string)
        for node in ast.walk(tree):
            if isinstance(node, ast.Assert):
                try:
                    assertions.append(ast.unparse(node.test).strip())
                except AttributeError:
                    assertions.append(f"Assertion involving node type: {type(node.test).__name__}")
    except SyntaxError:
        pass
    return assertions

def baseline_evaluate_generated_tests(function_id, function_code, entry_point, generated_test_code, llm_model_name):
    evaluation = {
        "unique_id": function_id,
        "llm_model": llm_model_name,
        "generated_test_code": generated_test_code,
        "syntactic_validity": False,
        "extracted_assertions": [],
        "num_assertions": 0,
        "execution_passes": None,
        "execution_error": None
    }
    if not generated_test_code or not isinstance(generated_test_code, str) or not generated_test_code.strip():
        evaluation["execution_error"] = "Empty or invalid generated test code."
        return evaluation

    try:
        ast.parse(generated_test_code)
        evaluation["syntactic_validity"] = True
    except SyntaxError as e:
        evaluation["execution_error"] = f"SyntaxError: {e}"
        return evaluation

    evaluation["extracted_assertions"] = extract_assertions_from_code(generated_test_code)
    evaluation["num_assertions"] = len(evaluation["extracted_assertions"])

    if evaluation["syntactic_validity"] and entry_point and isinstance(function_code, str) and function_code.strip():
        # Sanitize names for file operations
        safe_function_id = re.sub(r"[^a-zA-Z0-9_]", "", str(function_id))
        safe_llm_name = re.sub(r"[^a-zA-Z0-9_]", "", llm_model_name)
        temp_module_name = f"temp_test_module_{safe_function_id}_{safe_llm_name}"

        full_code_to_execute = function_code + "\n\n" + generated_test_code

        if not re.search(r"if __name__ == .__main__.:", generated_test_code, re.IGNORECASE) and "pytest" not in generated_test_code.lower():
            test_function_calls = []
            try:
                parsed_tests = ast.parse(generated_test_code)
                for node in parsed_tests.body:
                    if isinstance(node, ast.FunctionDef) and node.name.startswith("test"):
                        test_function_calls.append(f"{node.name}()")
                if test_function_calls:
                    full_code_to_execute += "\n\n# Auto-added test runner\n" + "\n".join(test_function_calls)
            except SyntaxError:
                pass

        try:
            with open(f"{temp_module_name}.py", "w", encoding="utf-8") as f:
                f.write(full_code_to_execute)

            import subprocess
            process = subprocess.run(["python", f"{temp_module_name}.py"], capture_output=True, text=True, timeout=20, encoding="utf-8", errors="replace")

            if process.returncode == 0:
                evaluation["execution_passes"] = True
            else:
                evaluation["execution_passes"] = False
                evaluation["execution_error"] = process.stderr[:1000]
        except FileNotFoundError:
            evaluation["execution_error"] = "Python executable not found for subprocess execution."
            evaluation["execution_passes"] = False
        except subprocess.TimeoutExpired:
            evaluation["execution_error"] = "Test execution timed out."
            evaluation["execution_passes"] = False
        except Exception as e:
            evaluation["execution_error"] = f"Runtime error: {str(e)[:500]}"
            evaluation["execution_passes"] = False
        finally:
            if os.path.exists(f"{temp_module_name}.py"):
                os.remove(f"{temp_module_name}.py")
    elif not isinstance(function_code, str) or not function_code.strip():
        evaluation["execution_error"] = "Original function code is missing or empty."
    elif not entry_point:
        evaluation["execution_error"] = "Function entry point is missing."

    return evaluation

# --- Main Processing Function ---
def run_llm_test_generation_and_evaluation(BASE_DRIVE_PATH):
    print("\n--- Starting LLM Test Generation and Baseline Evaluation ---")

    llm_models_to_use = {
        "DeepSeek-Coder-1.3B-instruct": {"id": "deepseek-ai/deepseek-coder-1.3b-instruct", "quant": True},
        "Phi-2": {"id": "microsoft/phi-2", "quant": True}
    }

    loaded_llms = {}
    for name, config in llm_models_to_use.items():
        llm_pipe = load_llm_model(name, config["id"], use_quantization=config["quant"])
        if llm_pipe:
            loaded_llms[name] = llm_pipe

    if not loaded_llms:
        print("No LLMs were loaded successfully. Aborting test generation.")
        return

    combined_dataset_path = os.path.join(BASE_DRIVE_PATH, "combined_dataset.csv")
    if not os.path.exists(combined_dataset_path):
        print(f"Combined dataset not found at {combined_dataset_path}. Please run data collection first.")
        return

    try:
        combined_df = pd.read_csv(combined_dataset_path)
        print(f"Loaded combined dataset with {len(combined_df)} functions.")
    except Exception as e:
        print(f"Error loading combined_dataset.csv: {e}")
        return

    prompt_template = """You are an expert Python programmer. Given the following Python function, please generate a comprehensive suite of pytest-style unit tests. Ensure your tests cover various edge cases and typical scenarios. The tests should include clear assertion statements. Do not include the original function in your response, only the test code. Wrap the test code in ```python ... ```.\n\nFunction Code:\n```python\n{function_code}\n```\n\nFunction Name: `{function_name}`\n\nPlease provide the pytest unit tests below:\n```python\n# Your pytest unit tests here\n"""

    all_generated_tests_data = []
    baseline_evaluation_results = []

    # Consider processing a subset for faster runs in Colab during development
    # num_functions_to_process = min(len(combined_df), 10)
    # print(f"Processing a subset of {num_functions_to_process} functions for test generation.")
    # combined_df_subset = combined_df.sample(n=num_functions_to_process, random_state=42) if len(combined_df) > num_functions_to_process else combined_df
    combined_df_subset = combined_df # Process all for full run

    for index, row in combined_df_subset.iterrows():
        unique_id = row["unique_id"]
        function_code = str(row["function_code"])
        entry_point = str(row.get("entry_point", ""))

        if not function_code.strip() or not entry_point.strip():
            print(f"Skipping function {unique_id} due to missing code or entry point.")
            baseline_evaluation_results.append({
                "unique_id": unique_id, "llm_model": "N/A", "generated_test_code": None,
                "syntactic_validity": False, "extracted_assertions": [], "num_assertions": 0,
                "execution_passes": None, "execution_error": "Missing function code or entry point"
            })
            continue

        print(f"\nProcessing function: {unique_id} ({entry_point})")
        for llm_name, llm_pipe in loaded_llms.items():
            generated_test_code = generate_tests_with_llm(llm_pipe, function_code, entry_point, prompt_template)

            all_generated_tests_data.append({
                "unique_id": unique_id,
                "function_code": function_code,
                "entry_point": entry_point,
                "llm_model": llm_name,
                "generated_test_code": generated_test_code if generated_test_code else ""
            })

            evaluation_metrics = baseline_evaluate_generated_tests(unique_id, function_code, entry_point, generated_test_code, llm_name)
            baseline_evaluation_results.append(evaluation_metrics)
            time.sleep(0.5)

    output_dir = os.path.join(BASE_DRIVE_PATH, "llm_generated_tests")
    os.makedirs(output_dir, exist_ok=True)

    generated_tests_df = pd.DataFrame(all_generated_tests_data)
    generated_tests_path = os.path.join(output_dir, "llm_generated_tests_output.csv")
    generated_tests_df.to_csv(generated_tests_path, index=False)
    print(f"\nLLM generated tests saved to {generated_tests_path}")
    if not generated_tests_df.empty: display(generated_tests_df.head())

    baseline_eval_df = pd.DataFrame(baseline_evaluation_results)
    baseline_eval_path = os.path.join(output_dir, "baseline_evaluation_summary.csv")
    baseline_eval_df.to_csv(baseline_eval_path, index=False)
    print(f"Baseline evaluation summary saved to {baseline_eval_path}")
    if not baseline_eval_df.empty: display(baseline_eval_df.head())

    print("\n--- LLM Test Generation and Baseline Evaluation Completed ---")

In [ ]:
run_llm_test_generation_and_evaluation(BASE_DRIVE_PATH)

## 4. Assertion Extraction and Baseline Evaluation

## 5. Mutation Testing

Mutation testing is used to assess the fault-detection capability of the generated assertions. We generate mutants of the original functions and check if the assertions can "kill" (detect) these mutants.


In [ ]:
# Implementation of Mutation Testing
import ast
import random
import re
import subprocess
import sys
import os
import pandas as pd
import time
from IPython.display import display

# --- Enhanced Mutation Operators ---
def mutate_comparison_operator(node):
    """Mutates comparison operators (e.g., > to <=, == to !=)."""
    if isinstance(node, ast.Compare):
        original_op = type(node.ops[0])
        mutated = False
        # More robust mutations: > to <=, < to >=, == to !=, etc.
        if original_op == ast.Gt: node.ops[0] = ast.LtE(); mutated = True
        elif original_op == ast.Lt: node.ops[0] = ast.GtE(); mutated = True
        elif original_op == ast.GtE: node.ops[0] = ast.Lt(); mutated = True
        elif original_op == ast.LtE: node.ops[0] = ast.Gt(); mutated = True
        elif original_op == ast.Eq: node.ops[0] = ast.NotEq(); mutated = True
        elif original_op == ast.NotEq: node.ops[0] = ast.Eq(); mutated = True
        elif original_op == ast.Is: node.ops[0] = ast.IsNot(); mutated = True
        elif original_op == ast.IsNot: node.ops[0] = ast.Is(); mutated = True
        return mutated
    return False

def mutate_arithmetic_operator(node):
    """Mutates arithmetic operators (e.g., + to -, * to /)."""
    if isinstance(node, ast.BinOp):
        original_op = type(node.op)
        mutated = False
        if original_op == ast.Add: node.op = ast.Sub(); mutated = True
        elif original_op == ast.Sub: node.op = ast.Add(); mutated = True
        elif original_op == ast.Mult: node.op = ast.Div(); mutated = True
        elif original_op == ast.Div: node.op = ast.Mult(); mutated = True
        return mutated
    return False

def mutate_logical_operator(node):
    """Mutates logical operators (e.g., and to or, or to and)."""
    if isinstance(node, ast.BoolOp):
        original_op = type(node.op)
        mutated = False
        if original_op == ast.And: node.op = ast.Or(); mutated = True
        elif original_op == ast.Or: node.op = ast.And(); mutated = True
        return mutated
    elif isinstance(node, ast.UnaryOp) and isinstance(node.op, ast.Not):
        # Remove the Not operation by replacing with the operand
        return True
    return False

def mutate_statement_deletion(node):
    """Modifies or effectively deletes statements like returns or assignments."""
    if isinstance(node, ast.Return) and hasattr(node, 'value') and node.value:
        # Modify return values
        if isinstance(node.value, ast.Constant):
            if isinstance(node.value.value, bool):
                node.value.value = not node.value.value
                return True
            elif isinstance(node.value.value, (int, float)):
                node.value.value = 0  # Replace with default/zero value
                return True
            elif isinstance(node.value.value, str):
                node.value.value = ""  # Replace with empty string
                return True
    elif isinstance(node, ast.Assign):
        # Modify assignments with constants
        if len(node.targets) == 1 and isinstance(node.value, ast.Constant):
            if isinstance(node.value.value, (int, float)):
                node.value.value = 0
                return True
            elif isinstance(node.value.value, bool):
                node.value.value = not node.value.value
                return True
            elif isinstance(node.value.value, str):
                node.value.value = ""
                return True
    return False

def mutate_boundary_value(node):
    """Mutates constant numeric values with boundary-focused mutations."""
    if isinstance(node, ast.Constant) and isinstance(node.value, (int, float)):
        original_value = node.value

        # Special cases for common boundaries
        if original_value == 1:
            node.value = 0 if random.random() < 0.5 else 2
        elif original_value == 0:
            node.value = -1 if random.random() < 0.5 else 1
        elif original_value > 0:
            # For positive values, consider boundary cases
            choices = [-original_value, original_value-1, original_value+1, 0]
            node.value = random.choice(choices)
        else:  # original_value < 0
            choices = [-original_value, original_value-1, original_value+1, 0]
            node.value = random.choice(choices)

        return True
    return False

# Combine all mutation operators
MUTATION_OPERATORS = [
    mutate_comparison_operator,
    mutate_arithmetic_operator,
    mutate_logical_operator,
    mutate_statement_deletion,
    mutate_boundary_value
]

# --- Mutation Generation Process ---
def generate_mutants(original_code_str, num_mutants_per_function=5):
    """Generate mutants with improved systematic approach and filtering."""
    mutants = []
    stats = {
        "requested": num_mutants_per_function,
        "generated": 0,
        "attempts": 0,
        "equivalent_detected": 0,
        "syntax_errors": 0
    }

    if not original_code_str or not isinstance(original_code_str, str) or not original_code_str.strip():
        return mutants, stats

    try:
        original_tree = ast.parse(original_code_str)
    except SyntaxError:
        stats["syntax_errors"] += 1
        return mutants, stats

    generated_mutant_codes = set()
    max_attempts = num_mutants_per_function * 15  # More attempts to find good mutants

    while len(mutants) < num_mutants_per_function and stats["attempts"] < max_attempts:
        stats["attempts"] += 1

        mutant_tree = ast.parse(original_code_str)  # Re-parse for a fresh tree
        nodes_in_mutant_tree = [n for n in ast.walk(mutant_tree)]
        random.shuffle(nodes_in_mutant_tree)

        mutation_applied = False
        for node_to_mutate in nodes_in_mutant_tree:
            # Try each operator in random order
            shuffled_operators = random.sample(MUTATION_OPERATORS, len(MUTATION_OPERATORS))
            for operator_func in shuffled_operators:
                if operator_func(node_to_mutate):
                    mutation_applied = True
                    break
            if mutation_applied:
                break

        if mutation_applied:
            try:
                mutant_code = ast.unparse(mutant_tree)

                # Check if this is an equivalent mutant or duplicate
                if mutant_code == original_code_str:
                    stats["equivalent_detected"] += 1
                    continue

                if mutant_code not in generated_mutant_codes:
                    # Verify the mutated code is still valid Python
                    ast.parse(mutant_code)
                    mutants.append(mutant_code)
                    generated_mutant_codes.add(mutant_code)
                    stats["generated"] += 1
            except SyntaxError:
                stats["syntax_errors"] += 1
            except Exception:
                pass  # Other errors during generation

    return mutants, stats

# --- Test Execution against Mutants ---
def run_tests_on_mutant(function_entry_point, mutant_code_str, test_code_str, original_function_code_str):
    """Execute tests against a mutated function and check if the tests kill the mutant."""
    if not test_code_str or not isinstance(test_code_str, str) or not test_code_str.strip() or \
       not mutant_code_str or not isinstance(mutant_code_str, str) or not mutant_code_str.strip():
        return False

    full_script_content = mutant_code_str + "\n\n" + test_code_str

    if not re.search(r"if __name__ == .__main__.:", test_code_str, re.IGNORECASE) and "pytest" not in test_code_str.lower():
        test_function_calls = []
        try:
            parsed_tests = ast.parse(test_code_str)
            for node in parsed_tests.body:
                if isinstance(node, ast.FunctionDef) and node.name.startswith("test"):
                    test_function_calls.append(f"{node.name}()")
            if test_function_calls:
                full_script_content += "\n\n# Auto-added test runner\n" + "\n".join(test_function_calls)
        except SyntaxError:
            pass

    # Sanitize names for file operations
    safe_entry_point = re.sub(r"[^a-zA-Z0-9_]", "", str(function_entry_point))
    temp_script_path = f"temp_mutant_test_runner_{safe_entry_point}.py"

    try:
        with open(temp_script_path, "w", encoding="utf-8") as f:
            f.write(full_script_content)

        process = subprocess.run([sys.executable, temp_script_path],
                                capture_output=True, text=True, timeout=20,
                                encoding="utf-8", errors="replace")

        # Mutant is killed if tests fail (non-zero return code)
        return process.returncode != 0

    except subprocess.TimeoutExpired:
        return True  # Timeout indicates infinite loop - consider killed
    except Exception:
        return False  # Other errors in execution - consider survived
    finally:
        if os.path.exists(temp_script_path):
            try:
                os.remove(temp_script_path)
            except:
                pass

# --- External Mutation Tool Integration ---
def run_external_mutation_tool(function_file, test_file):
    """Run mutation testing using external tools like mutmut or MutPy."""
    results = {"success": False, "tool": None, "score": 0, "details": {}}

    # Try mutmut first
    try:
        import subprocess
        print("Attempting to run mutmut...")

        with open(function_file, "w") as f:
            f.write("# Function file for mutation testing")

        with open(test_file, "w") as f:
            f.write("# Test file for mutation testing")

        result = subprocess.run(
            ["mutmut", "run", "--paths-to-mutate", function_file, "--test-command", f"python -m pytest {test_file}"],
            capture_output=True, text=True, timeout=300
        )

        if result.returncode == 0 or "mutants" in result.stdout:
            results["success"] = True
            results["tool"] = "mutmut"
            # Extract score from output
            results["details"]["output"] = result.stdout

            # Very basic parsing - would need to be refined
            if "killed" in result.stdout and "survived" in result.stdout:
                killed_match = re.search(r"(\d+) killed", result.stdout)
                survived_match = re.search(r"(\d+) survived", result.stdout)

                if killed_match and survived_match:
                    killed = int(killed_match.group(1))
                    survived = int(survived_match.group(1))
                    total = killed + survived

                    if total > 0:
                        results["score"] = (killed / total) * 100

            return results
    except Exception as e:
        print(f"mutmut failed: {e}")

    # Fallback to MutPy if mutmut fails
    try:
        print("Attempting to run MutPy...")
        result = subprocess.run(
            ["mut.py", "--target", function_file, "--unit-test", test_file],
            capture_output=True, text=True, timeout=300
        )

        if result.returncode == 0 or "score" in result.stdout:
            results["success"] = True
            results["tool"] = "MutPy"
            results["details"]["output"] = result.stdout

            # Extract score from output
            score_match = re.search(r"Mutation score.*?(\d+\.\d+)%", result.stdout)
            if score_match:
                results["score"] = float(score_match.group(1))

            return results
    except Exception as e:
        print(f"MutPy failed: {e}")

    print("External mutation tools unavailable or failed.")
    return results

# --- Main Mutation Testing Function ---
def run_mutation_testing_for_llm_tests(BASE_DRIVE_PATH):
    print("\n--- Starting Enhanced Mutation Testing for LLM-Generated Tests ---")

    llm_tests_output_path = os.path.join(BASE_DRIVE_PATH, "llm_generated_tests", "llm_generated_tests_output.csv")
    if not os.path.exists(llm_tests_output_path):
        print(f"LLM generated tests output not found at {llm_tests_output_path}. Run LLM test generation first.")
        return

    try:
        llm_tests_df = pd.read_csv(llm_tests_output_path)
        print(f"Loaded {len(llm_tests_df)} LLM-generated test instances.")
    except Exception as e:
        print(f"Error loading LLM generated tests: {e}")
        return

    mutation_results = []
    mutation_statistics = []
    num_mutants_per_func = 7  # Increased from 5 to 7 for more comprehensive testing

    # Process all instances or use a subset for testing
    llm_tests_df_subset = llm_tests_df

    for index, row in llm_tests_df_subset.iterrows():
        unique_id = row["unique_id"]
        original_function_code = str(row["function_code"])
        llm_generated_test_code = str(row["generated_test_code"])
        llm_model_name = row["llm_model"]
        entry_point = str(row.get("entry_point", ""))

        if (pd.isna(original_function_code) or pd.isna(llm_generated_test_code) or
                not original_function_code.strip() or not llm_generated_test_code.strip() or not entry_point.strip()):
            mutation_results.append({
                "unique_id": unique_id,
                "llm_model": llm_model_name,
                "total_mutants_generated": 0,
                "mutants_killed": 0,
                "mutation_score": 0.0,
                "error": "Missing function, test code, or entry point"
            })
            mutation_statistics.append({
                "unique_id": unique_id,
                "llm_model": llm_model_name,
                "requested": num_mutants_per_func,
                "generated": 0,
                "attempts": 0,
                "equivalent_detected": 0,
                "syntax_errors": 0
            })
            continue

        print(f"\nRunning mutation testing for function {unique_id} ({entry_point}), tests by {llm_model_name}...")
        mutants, stats = generate_mutants(original_function_code, num_mutants_per_function=num_mutants_per_func)
        stats["unique_id"] = unique_id
        stats["llm_model"] = llm_model_name
        mutation_statistics.append(stats)

        if not mutants:
            print(f"No valid mutants generated for {unique_id} after {stats['attempts']} attempts.")
            mutation_results.append({
                "unique_id": unique_id,
                "llm_model": llm_model_name,
                "total_mutants_generated": 0,
                "mutants_killed": 0,
                "mutation_score": 0.0,
                "error": f"No valid mutants generated after {stats['attempts']} attempts"
            })
            continue

        mutants_killed = 0
        killed_details = []
        survived_mutations = {}

        print(f"Generated {len(mutants)} valid mutants for {unique_id}. Testing each...")

        for i, mutant_code in enumerate(mutants):
            killed = run_tests_on_mutant(entry_point, mutant_code, llm_generated_test_code, original_function_code)
            if killed:
                mutants_killed += 1
                killed_details.append(i)
            else:
                # Track types of survived mutations
                if "==" in mutant_code and "!=" in original_function_code:
                    survived_mutations["comparison_operators"] = survived_mutations.get("comparison_operators", 0) + 1
                elif "+" in mutant_code and "-" in original_function_code:
                    survived_mutations["arithmetic_operators"] = survived_mutations.get("arithmetic_operators", 0) + 1
                elif "and" in mutant_code and "or" in original_function_code:
                    survived_mutations["logical_operators"] = survived_mutations.get("logical_operators", 0) + 1
                else:
                    survived_mutations["other"] = survived_mutations.get("other", 0) + 1

        mutation_score = (mutants_killed / len(mutants)) * 100 if mutants else 0.0
        print(f"Mutation testing for {unique_id} by {llm_model_name}: {mutants_killed}/{len(mutants)} killed. Score: {mutation_score:.2f}%")

        mutation_results.append({
            "unique_id": unique_id,
            "llm_model": llm_model_name,
            "original_function_code": original_function_code,
            "llm_generated_test_code": llm_generated_test_code,
            "total_mutants_generated": len(mutants),
            "mutants_killed": mutants_killed,
            "mutation_score": mutation_score,
            "killed_mutant_indices": str(killed_details),
            "survived_mutations": str(survived_mutations),
            "error": None
        })

        time.sleep(0.2)  # Small delay

    # Save detailed results
    mutation_results_df = pd.DataFrame(mutation_results)
    mutation_stats_df = pd.DataFrame(mutation_statistics)

    output_dir = os.path.join(BASE_DRIVE_PATH, "mutation_testing_results")
    os.makedirs(output_dir, exist_ok=True)

    mutation_results_path = os.path.join(output_dir, "mutation_testing_summary.csv")
    mutation_stats_path = os.path.join(output_dir, "mutation_generation_statistics.csv")

    mutation_results_df.to_csv(mutation_results_path, index=False)
    mutation_stats_df.to_csv(mutation_stats_path, index=False)

    print(f"\nMutation testing results saved to {mutation_results_path}")
    print(f"Mutation generation statistics saved to {mutation_stats_path}")

    if not mutation_results_df.empty:
        display(mutation_results_df.head())

        # Calculate and display summary statistics by LLM model
        model_summary = mutation_results_df.groupby("llm_model").agg({
            "total_mutants_generated": "sum",
            "mutants_killed": "sum",
            "mutation_score": "mean"
        }).reset_index()
        model_summary["overall_kill_rate"] = (
            model_summary["mutants_killed"] / model_summary["total_mutants_generated"] * 100
        ).round(2)

        print("\nMutation Testing Summary by LLM Model:")
        display(model_summary)

    print("\n--- Enhanced Mutation Testing for LLM-Generated Tests Completed ---")



In [ ]:
run_mutation_testing_for_llm_tests(BASE_DRIVE_PATH)

Streaming output truncated to the last 5000 lines.
Running mutation testing for function HumanEval+_HumanEval/13 (greatest_common_divisor), tests by Phi-2...
Generated 3 valid mutants for HumanEval+_HumanEval/13. Testing each...
Mutation testing for HumanEval+_HumanEval/13 by Phi-2: 3/3 killed. Score: 100.00%

Running mutation testing for function HumanEval+_HumanEval/14 (all_prefixes), tests by DeepSeek-Coder-1.3B-instruct...
Generated 3 valid mutants for HumanEval+_HumanEval/14. Testing each...
Mutation testing for HumanEval+_HumanEval/14 by DeepSeek-Coder-1.3B-instruct: 3/3 killed. Score: 100.00%

Running mutation testing for function HumanEval+_HumanEval/14 (all_prefixes), tests by Phi-2...
Generated 3 valid mutants for HumanEval+_HumanEval/14. Testing each...
Mutation testing for HumanEval+_HumanEval/14 by Phi-2: 0/3 killed. Score: 0.00%

Running mutation testing for function HumanEval+_HumanEval/15 (string_sequence), tests by DeepSeek-Coder-1.3B-instruct...
Generated 3 valid muta

,unique_id,llm_model,original_function_code,llm_generated_test_code,total_mutants_generated,mutants_killed,mutation_score,killed_mutant_indices,survived_mutations,error
0,HumanEval+_HumanEval/0,DeepSeek-Coder-1.3B-instruct,from typing import List\n\n\ndef has_close_ele...,import pytest\nfrom typing import List\n\ndef ...,7,0,0.000000,[],{'arithmetic_operators': 7},None
1,HumanEval+_HumanEval/0,Phi-2,from typing import List\n\n\ndef has_close_ele...,# Your pytest unit tests here\ndef test_has_cl...,7,0,0.000000,[],"{'arithmetic_operators': 6, 'other': 1}",None
2,HumanEval+_HumanEval/1,DeepSeek-Coder-1.3B-instruct,from typing import List\n\n\ndef separate_pare...,# Your pytest unit tests here\nimport pytest\n...,7,0,0.000000,[],{'comparison_operators': 7},None
3,HumanEval+_HumanEval/1,Phi-2,from typing import List\n\n\ndef separate_pare...,# Test Cases\ndef test_separate_paren_groups_s...,7,6,85.714286,"[0, 1, 2, 4, 5, 6]",{'comparison_operators': 1},None
4,HumanEval+_HumanEval/2,DeepSeek-Coder-1.3B-instruct,def truncate_number(number: float) -> float:\n...,import pytest\nfrom your_module import truncat...,1,1,100.000000,[0],{},None



Mutation Testing Summary by LLM Model:


,llm_model,total_mutants_generated,mutants_killed,mutation_score,overall_kill_rate
0,DeepSeek-Coder-1.3B-instruct,3163,1661,44.316552,52.51
1,Phi-2,3163,966,24.827883,30.54



--- Enhanced Mutation Testing for LLM-Generated Tests Completed ---


In [ ]:
# Implementation of Assertion Refinement Dataset Creation

def analyze_assertions_for_refinement(BASE_DRIVE_PATH):
    """Analyze assertion quality and create a dataset for refinement."""
    print("\n--- Creating Assertion Refinement Dataset ---")

    mutation_results_path = os.path.join(BASE_DRIVE_PATH, "mutation_testing_results", "mutation_testing_summary.csv")
    baseline_eval_path = os.path.join(BASE_DRIVE_PATH, "llm_generated_tests", "baseline_evaluation_summary.csv")

    if not os.path.exists(mutation_results_path) or not os.path.exists(baseline_eval_path):
        print("Required data for assertion analysis not found. Run mutation testing first.")
        return None

    try:
        mutation_df = pd.read_csv(mutation_results_path)
        baseline_df = pd.read_csv(baseline_eval_path)

        # Merge for comprehensive analysis
        analysis_df = pd.merge(
            baseline_df[['unique_id', 'llm_model', 'execution_passes', 'syntactic_validity', 'num_assertions', 'extracted_assertions']],
            mutation_df[['unique_id', 'llm_model', 'mutation_score', 'mutants_killed', 'total_mutants_generated']],
            on=['unique_id', 'llm_model'],
            how='inner'
        )

        print(f"Loaded data for {len(analysis_df)} test cases for assertion quality analysis.")
    except Exception as e:
        print(f"Error loading or merging data: {e}")
        return None

    # Classify assertions based on quality
    analysis_df['assertion_quality'] = 'unknown'

    # Classification logic:
    # 1. Strong assertions: High mutation score (kill most mutants)
    # 2. Weak assertions: Low mutation score (kill few mutants)
    # 3. Incorrect assertions: Fail on correct implementation

    analysis_df.loc[analysis_df['mutation_score'] >= 80, 'assertion_quality'] = 'strong'
    analysis_df.loc[(analysis_df['mutation_score'] < 80) & (analysis_df['mutation_score'] >= 30), 'assertion_quality'] = 'moderate'
    analysis_df.loc[analysis_df['mutation_score'] < 30, 'assertion_quality'] = 'weak'
    analysis_df.loc[analysis_df['execution_passes'] == False, 'assertion_quality'] = 'incorrect'

    # Extract and analyze individual assertions
    def extract_assertion_details(row):
        try:
            extracted = ast.literal_eval(row['extracted_assertions']) if isinstance(row['extracted_assertions'], str) else []
            if not extracted or not isinstance(extracted, list):
                return []

            details = []
            for assertion in extracted:
                # Basic classification of assertion types
                assertion_type = 'general'
                if 'assert isinstance' in assertion:
                    assertion_type = 'type_check'
                elif 'assert len' in assertion:
                    assertion_type = 'length_check'
                elif '==' in assertion:
                    assertion_type = 'equality'
                elif 'is not None' in assertion:
                    assertion_type = 'existence'
                elif 'is None' in assertion:
                    assertion_type = 'absence'
                elif 'in ' in assertion:
                    assertion_type = 'membership'
                elif 'not in ' in assertion:
                    assertion_type = 'exclusion'
                elif '>' in assertion or '<' in assertion:
                    assertion_type = 'comparison'

                details.append({
                    'text': assertion,
                    'type': assertion_type,
                    'quality': row['assertion_quality'],
                    'mutation_score': row['mutation_score']
                })
            return details
        except:
            return []

    all_assertion_details = []
    for _, row in analysis_df.iterrows():
        details = extract_assertion_details(row)
        for detail in details:
            detail['unique_id'] = row['unique_id']
            detail['llm_model'] = row['llm_model']
            all_assertion_details.append(detail)

    assertion_details_df = pd.DataFrame(all_assertion_details)

    # Split into training, validation, and test sets
    if len(assertion_details_df) > 0:
        # Shuffle the DataFrame
        assertion_details_df = assertion_details_df.sample(frac=1, random_state=42).reset_index(drop=True)

        # Calculate split indices
        train_end = int(len(assertion_details_df) * 0.7)
        val_end = int(len(assertion_details_df) * 0.85)

        # Split the DataFrame
        train_df = assertion_details_df.iloc[:train_end].copy()
        val_df = assertion_details_df.iloc[train_end:val_end].copy()
        test_df = assertion_details_df.iloc[val_end:].copy()

        # Save the splits
        output_dir = os.path.join(BASE_DRIVE_PATH, "assertion_refinement_dataset")
        os.makedirs(output_dir, exist_ok=True)

        train_df.to_csv(os.path.join(output_dir, "train.csv"), index=False)
        val_df.to_csv(os.path.join(output_dir, "validation.csv"), index=False)
        test_df.to_csv(os.path.join(output_dir, "test.csv"), index=False)

        # Also save the full dataset
        assertion_details_df.to_csv(os.path.join(output_dir, "full_dataset.csv"), index=False)

        print(f"Assertion refinement dataset created with {len(train_df)} training, {len(val_df)} validation, and {len(test_df)} test samples.")
        print(f"Dataset saved to {output_dir}")

        # Summary statistics
        print("\nAssertion Quality Distribution:")
        quality_counts = analysis_df['assertion_quality'].value_counts()
        for quality, count in quality_counts.items():
            print(f"  {quality}: {count} ({count/len(analysis_df)*100:.1f}%)")

        print("\nAssertion Type Distribution:")
        type_counts = assertion_details_df['type'].value_counts()
        for type_, count in type_counts.items():
            print(f"  {type_}: {count} ({count/len(assertion_details_df)*100:.1f}%)")

        return assertion_details_df
    else:
        print("No assertion details could be extracted for the refinement dataset.")
        return None


In [ ]:
assertion_dataset = analyze_assertions_for_refinement(BASE_DRIVE_PATH)

In [ ]:
# Implementation of Assertion Refinement Techniques

def improve_assertion_with_pattern_based_refinement(assertion_text, assertion_type):
    """
    Implements pattern-based refinement to improve weak assertions.
    This is a rule-based approach that replaces weak patterns with stronger alternatives.
    """
    improved_assertion = assertion_text

    # Pattern 1: Replace simple existence checks with more specific assertions
    if assertion_type == 'existence' and 'is not None' in assertion_text:
        # Extract the expression being checked
        expr = assertion_text.split('assert ')[1].split(' is not None')[0].strip()
        if expr:
            improved_assertion = f"assert isinstance({expr}, (list, dict, str)), 'Expected {expr} to be a container type'"

    # Pattern 2: Enhance equality checks with type checks
    elif assertion_type == 'equality' and '==' in assertion_text and 'isinstance' not in assertion_text:
        parts = assertion_text.split('==')
        if len(parts) == 2:
            left = parts[0].replace('assert', '').strip()
            right = parts[1].strip()
            if left and right and not right.isdigit():  # Don't add type check for numeric constants
                improved_assertion = f"assert isinstance({left}, type({right})), 'Type mismatch'\nassert {left} == {right}, 'Value mismatch'"

    # Pattern 3: Add boundary checks to numeric assertions
    elif assertion_type == 'comparison' and ('>' in assertion_text or '<' in assertion_text):
        # Example: assert x > 0 -> assert x > 0, f"Expected {x} to be positive"
        parts = re.split(r'[<>]=?', assertion_text)
        if len(parts) == 2:
            expr = parts[0].replace('assert', '').strip()
            boundary = parts[1].strip()
            if expr and boundary:
                # Extract the comparison operator
                operator = re.search(r'([<>]=?)', assertion_text).group(1)
                # Add a descriptive message
                if operator == '>':
                    message = f"Expected {expr} to be greater than {boundary}"
                elif operator == '>=':
                    message = f"Expected {expr} to be at least {boundary}"
                elif operator == '<':
                    message = f"Expected {expr} to be less than {boundary}"
                elif operator == '<=':
                    message = f"Expected {expr} to be at most {boundary}"

                improved_assertion = f"assert {expr} {operator} {boundary}, '{message}'"

    # Pattern 4: Enhance collection checks
    elif assertion_type == 'length_check' and 'len(' in assertion_text:
        expr = re.search(r'len\((.*?)\)', assertion_text).group(1)
        if expr:
            # Add type check before length check
            if '==' in assertion_text:
                length = assertion_text.split('==')[1].strip()
                improved_assertion = f"assert isinstance({expr}, (list, tuple, dict, str)), '{expr} should be a collection'\nassert len({expr}) == {length}, '{expr} should have length {length}'"
            elif '>' in assertion_text:
                length = assertion_text.split('>')[1].strip()
                improved_assertion = f"assert isinstance({expr}, (list, tuple, dict, str)), '{expr} should be a collection'\nassert len({expr}) > {length}, '{expr} should have length greater than {length}'"

    # Pattern 5: Add descriptive messages to assertions that don't have them
    if ',' not in improved_assertion and not improved_assertion.endswith(')'):
        improved_assertion += f", 'Assertion failed: {improved_assertion.replace('assert ', '')}'"

    return improved_assertion

def improve_assertion_with_semantic_refinement(assertion_text, function_code, context=None):
    """
    Implements semantic refinement to generate more meaningful assertions.
    This analyzes function purpose to create assertions that better capture intended behavior.
    """
    # Parse the function to understand its purpose
    try:
        # Parse both function code and existing assertion
        function_ast = ast.parse(function_code)
        assertion_ast = ast.parse(assertion_text)

        function_name = None
        return_type = None
        function_args = []

        # Extract function details
        for node in ast.walk(function_ast):
            if isinstance(node, ast.FunctionDef):
                function_name = node.name
                function_args = [arg.arg for arg in node.args.args]

                # Check for return type hints if available
                if node.returns:
                    return_type = ast.unparse(node.returns)

                # Check docstring for return type info
                if node.body and isinstance(node.body[0], ast.Expr) and isinstance(node.body[0].value, ast.Constant):
                    docstring = node.body[0].value.value
                    if "return" in docstring.lower():
                        # Simple regex to find return type in docstring
                        match = re.search(r"return[s]?:?\s*([\w\[\]]+)", docstring.lower())
                        if match:
                            return_type = match.group(1)

                # Analyze return statements to infer return type
                for subnode in ast.walk(node):
                    if isinstance(subnode, ast.Return) and subnode.value:
                        if isinstance(subnode.value, ast.List):
                            return_type = "list"
                        elif isinstance(subnode.value, ast.Dict):
                            return_type = "dict"
                        elif isinstance(subnode.value, ast.Str):
                            return_type = "str"
                        elif isinstance(subnode.value, ast.Num):
                            return_type = "number"
                        elif isinstance(subnode.value, ast.NameConstant) and subnode.value.value in [True, False]:
                            return_type = "bool"
                break

        # Semantic improvements based on function analysis
        improved_assertion = assertion_text

        # If we have type information, use it to improve the assertion
        if return_type:
            if return_type == "list" and "isinstance" not in improved_assertion:
                # Add appropriate checks for list return type
                if any(arg in assertion_text for arg in function_args):
                    func_call = re.search(r'assert\s+([\w\._]+\(.*?\))', assertion_text)
                    if func_call:
                        func_expr = func_call.group(1)
                        improved_assertion = f"assert isinstance({func_expr}, list), 'Expected a list return value'\n"
                        improved_assertion += f"assert len({func_expr}) > 0, 'Expected non-empty list'\n"
                        improved_assertion += assertion_text

            elif return_type == "dict" and "isinstance" not in improved_assertion:
                # Add appropriate checks for dictionary return type
                if any(arg in assertion_text for arg in function_args):
                    func_call = re.search(r'assert\s+([\w\._]+\(.*?\))', assertion_text)
                    if func_call:
                        func_expr = func_call.group(1)
                        improved_assertion = f"assert isinstance({func_expr}, dict), 'Expected a dict return value'\n"
                        improved_assertion += assertion_text

            elif return_type == "bool" and "==" not in improved_assertion:
                # Make boolean assertions more explicit
                if "is True" in assertion_text or "is not False" in assertion_text:
                    func_call = re.search(r'assert\s+([\w\._]+\(.*?\))', assertion_text)
                    if func_call:
                        func_expr = func_call.group(1)
                        improved_assertion = f"assert {func_expr} == True, 'Expected function to return True'"

        # If assertion is checking exception but doesn't use pytest.raises
        if "try:" in assertion_text and "except" in assertion_text and "pytest" not in assertion_text:
            improved_assertion = "import pytest\n"
            exception_type = re.search(r'except\s+(\w+):', assertion_text)
            if exception_type:
                exception = exception_type.group(1)
                func_call = re.search(r'(\w+\(.*?\))', assertion_text)
                if func_call:
                    func_expr = func_call.group(1)
                    improved_assertion += f"with pytest.raises({exception}):\n    {func_expr}"

        return improved_assertion

    except SyntaxError:
        # If we can't parse the code, just return the original assertion
        return assertion_text

def apply_hybrid_assertion_refinement(assertion_text, function_code, original_quality='weak'):
    """
    Implements the hybrid approach combining pattern-based and semantic refinement.
    This two-stage process first applies pattern-based improvements then semantic enhancements.
    """
    # Identify the assertion type
    assertion_type = 'general'
    if 'assert isinstance' in assertion_text:
        assertion_type = 'type_check'
    elif 'assert len' in assertion_text:
        assertion_type = 'length_check'
    elif '==' in assertion_text:
        assertion_type = 'equality'
    elif 'is not None' in assertion_text:
        assertion_type = 'existence'
    elif 'is None' in assertion_text:
        assertion_type = 'absence'
    elif 'in ' in assertion_text:
        assertion_type = 'membership'
    elif 'not in ' in assertion_text:
        assertion_type = 'exclusion'
    elif '>' in assertion_text or '<' in assertion_text:
        assertion_type = 'comparison'

    # Stage 1: Apply pattern-based refinement
    pattern_improved = improve_assertion_with_pattern_based_refinement(assertion_text, assertion_type)

    # Stage 2: Apply semantic refinement if needed
    if pattern_improved == assertion_text or original_quality == 'weak':
        final_improved = improve_assertion_with_semantic_refinement(pattern_improved, function_code)
        return final_improved
    else:
        return pattern_improved

# Example usage function
def demonstrate_assertion_refinement(BASE_DRIVE_PATH):
    """Demonstrate the assertion refinement process with examples from the dataset."""
    print("\n--- Demonstrating Hybrid Assertion Refinement Approach ---")

    # Load the dataset if available
    dataset_path = os.path.join(BASE_DRIVE_PATH, "assertion_refinement_dataset", "full_dataset.csv")
    if os.path.exists(dataset_path):
        df = pd.read_csv(dataset_path)
        print(f"Loaded {len(df)} assertions from refinement dataset.")

        # Select some weak assertions for demonstration
        weak_assertions = df[df['quality'] == 'weak'].sample(min(5, len(df[df['quality'] == 'weak'])))

        if not weak_assertions.empty:
            print("\nExample Assertion Refinements:")
            for i, row in enumerate(weak_assertions.iterrows(), 1):
                _, assertion = row

                # Load the original function code
                original_code = "def example_func():\n    return True"  # Default placeholder


                # using the unique_id to locate it

                print(f"\nExample {i}:")
                print(f"Original assertion ({assertion['type']}): {assertion['text']}")

                # Apply the hybrid refinement approach
                improved = apply_hybrid_assertion_refinement(
                    assertion['text'],
                    original_code,
                    assertion['quality']
                )

                print(f"Improved assertion: {improved}")
        else:
            print("No weak assertions found in the dataset for demonstration.")
    else:
        print(f"Assertion refinement dataset not found at {dataset_path}.")
        print("Run the assertion analysis function first to create the dataset.")

        # Demonstrate with some example assertions
        print("\nDemonstrating with example assertions:")

        examples = [
            ("assert result is not None", "def get_user_data(user_id):\n    return {'id': user_id, 'name': 'Test'}", "existence"),
            ("assert len(items) > 0", "def fetch_items(category):\n    return ['item1', 'item2']", "length_check"),
            ("assert calculate_total(10, 20) == 30", "def calculate_total(a, b):\n    return a + b", "equality")
        ]

        for i, (assertion, function, type_) in enumerate(examples, 1):
            print(f"\nExample {i}:")
            print(f"Original assertion ({type_}): {assertion}")
            improved = apply_hybrid_assertion_refinement(assertion, function, 'weak')
            print(f"Improved assertion: {improved}")


In [ ]:
demonstrate_assertion_refinement(BASE_DRIVE_PATH)

In [ ]:
# Implementation of Advanced Assertion Correction with LLM

import pandas as pd
import os
import json
import ast
import time
import re
import random
import sys
import subprocess
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from IPython.display import display, Markdown

In [ ]:
# --- Helper: Extract Assertions with Context ---
def extract_assertions_with_context(code_string):
    """Extract assertions from code string with their line numbers, context, and type classification."""
    assertions = []
    if not isinstance(code_string, str):
        return assertions

    try:
        tree = ast.parse(code_string)
        for node in ast.walk(tree):
            if isinstance(node, ast.Assert):
                try:
                    assertion_text = ast.unparse(node.test).strip()
                    line_number = node.lineno

                    # Classify assertion type
                    assertion_type = "unknown"

                    # Check for equality assertions
                    if isinstance(node.test, ast.Compare) and any(isinstance(op, ast.Eq) for op in node.test.ops):
                        assertion_type = "equality"
                    # Check for type assertions
                    elif (isinstance(node.test, ast.Call) and
                          isinstance(node.test.func, ast.Name) and
                          node.test.func.id == "isinstance"):
                        assertion_type = "type_check"
                    # Check for membership assertions
                    elif isinstance(node.test, ast.Compare) and any(isinstance(op, ast.In) for op in node.test.ops):
                        assertion_type = "membership"
                    # Check for length/emptiness assertions
                    elif (isinstance(node.test, ast.Compare) and
                          isinstance(node.test.left, ast.Call) and
                          isinstance(node.test.left.func, ast.Name) and
                          node.test.left.func.id == "len"):
                        assertion_type = "length_check"

                    # Get surrounding context (the test function containing this assertion)
                    containing_function = None
                    for potential_parent in ast.walk(tree):
                        if (isinstance(potential_parent, ast.FunctionDef) and
                            potential_parent.lineno <= node.lineno and
                            hasattr(potential_parent, 'end_lineno') and
                            potential_parent.end_lineno >= node.lineno):
                            containing_function = potential_parent.name
                            break

                    assertions.append({
                        "text": assertion_text,
                        "line": line_number,
                        "type": assertion_type,
                        "containing_function": containing_function
                    })
                except AttributeError:
                    assertions.append({
                        "text": f"Assertion involving node type: {type(node.test).__name__}",
                        "line": node.lineno,
                        "type": "unknown",
                        "containing_function": None
                    })
    except SyntaxError:
        pass

    return assertions

def analyze_function_for_testability(function_code):
    """Analyzes the function to determine what aspects should be tested."""
    analysis = {
        "return_type": None,
        "parameters": [],
        "edge_cases": [],
        "complex_logic": False
    }

    try:
        tree = ast.parse(function_code)
        for node in ast.walk(tree):
            # Extract function parameters
            if isinstance(node, ast.FunctionDef):
                analysis["parameters"] = [arg.arg for arg in node.args.args]

                # Check for docstring that might provide hints
                if (node.body and isinstance(node.body[0], ast.Expr) and
                    isinstance(node.body[0].value, ast.Constant) and
                    isinstance(node.body[0].value.value, str)):
                    docstring = node.body[0].value.value
                    if "return" in docstring.lower():
                        # Try to extract return type from docstring
                        return_match = re.search(r"return[s]?:?\s+([\w\[\]]+)", docstring.lower())
                        if return_match:
                            analysis["return_type"] = return_match.group(1)

                # Detect complex logic - conditionals, loops, multiple returns
                has_conditionals = any(isinstance(n, (ast.If, ast.IfExp)) for n in ast.walk(node))
                has_loops = any(isinstance(n, (ast.For, ast.While)) for n in ast.walk(node))
                returns = [n for n in ast.walk(node) if isinstance(n, ast.Return)]

                analysis["complex_logic"] = has_conditionals or has_loops or len(returns) > 1

                # Find returns to determine return type if not already found
                for ret_node in returns:
                    if ret_node.value:
                        if isinstance(ret_node.value, ast.List):
                            analysis["return_type"] = "list"
                        elif isinstance(ret_node.value, ast.Dict):
                            analysis["return_type"] = "dict"
                        elif isinstance(ret_node.value, ast.Constant) and isinstance(ret_node.value.value, str):
                            analysis["return_type"] = "str"
                        elif isinstance(ret_node.value, ast.Constant) and isinstance(ret_node.value.value, (int, float)):
                            analysis["return_type"] = "numeric"
                        elif isinstance(ret_node.value, ast.Constant) and isinstance(ret_node.value.value, bool):
                            analysis["return_type"] = "bool"
                        # Add more type detections as needed

            # Identify potential edge cases from conditionals
            if isinstance(node, ast.If):
                if (isinstance(node.test, ast.Compare) and
                    any(isinstance(op, (ast.Eq, ast.NotEq, ast.Lt, ast.LtE, ast.Gt, ast.GtE))
                        for op in node.test.ops)):
                    if hasattr(node.test, 'comparators') and node.test.comparators:
                        comp = node.test.comparators[0]
                        if isinstance(comp, ast.Constant) and comp.value in (0, 1, "", [], {}, None):
                            analysis["edge_cases"].append(f"Value equals {comp.value}")

            # Check for error handling
            if isinstance(node, ast.Try):
                analysis["edge_cases"].append("Error handling")

    except SyntaxError:
        pass

    return analysis

def analyze_assertion_quality(assertions, function_code):
    """Analyze the quality of assertions relative to the function being tested."""
    analysis = {
        "total_assertions": len(assertions),
        "assertions_with_messages": sum(1 for a in assertions if a.get("has_message", False)),
        "weak_assertions": [],
        "highlights": []
    }

    # Identify weak assertion patterns
    weak_patterns = [
        r"is not None",
        r"is None",
        r"== True",
        r"== False",
        r"isinstance\(.*\)",
        r"in ",
        r"len\(.*\) [><=]"
    ]

    for assertion in assertions:
        assertion_text = assertion.get("assertion", "")
        for pattern in weak_patterns:
            if re.search(pattern, assertion_text):
                analysis["weak_assertions"].append(assertion)
                break

    # Generate highlights based on analysis
    if analysis["total_assertions"] == 0:
        analysis["highlights"].append("No assertions found in test code")
    else:
        if len(analysis["weak_assertions"]) > 0:
            analysis["highlights"].append(f"Found {len(analysis['weak_assertions'])} weak assertions that should be strengthened")

        if analysis["assertions_with_messages"] == 0:
            analysis["highlights"].append("No assertions have error messages")

        # Try to detect if assertions might be missing important function aspects
        try:
            func_ast = ast.parse(function_code)
            has_conditionals = any(isinstance(node, ast.If) for node in ast.walk(func_ast))
            has_loops = any(isinstance(node, (ast.For, ast.While)) for node in ast.walk(func_ast))

            if has_conditionals and analysis["total_assertions"] < 2:
                analysis["highlights"].append("Function contains conditional logic but may be undertested")
            if has_loops and analysis["total_assertions"] < 2:
                analysis["highlights"].append("Function contains loops but may be undertested")
        except:
            pass

    return analysis

def analyze_survived_mutations(function_code, test_code):
    """Analyze which types of mutations likely survived based on function and test code."""
    # This is a simplified placeholder. In a real implementation,

    survived = {
        "comparison_operators": 0,
        "arithmetic_operators": 0,
        "constant_values": 0,
        "logical_operators": 0
    }

    # Simple heuristic analysis - can be much more sophisticated
    try:
        func_ast = ast.parse(function_code)

        # Count potential mutation targets
        comparisons = sum(1 for _ in ast.walk(func_ast) if isinstance(_, ast.Compare))
        arithmetic = sum(1 for _ in ast.walk(func_ast) if isinstance(_, ast.BinOp) and
                        isinstance(_.op, (ast.Add, ast.Sub, ast.Mult, ast.Div)))
        constants = sum(1 for _ in ast.walk(func_ast) if isinstance(_, ast.Constant) and
                      isinstance(_.value, (int, float)))
        logical = sum(1 for _ in ast.walk(func_ast) if isinstance(_, ast.BoolOp))

        # Estimate survived mutations based on what's in the function vs what's tested
        test_ast = ast.parse(test_code)
        test_eq_count = sum(1 for _ in ast.walk(test_ast) if isinstance(_, ast.Compare) and
                          any(isinstance(op, ast.Eq) for op in getattr(_, 'ops', [])))

        survived["comparison_operators"] = max(0, comparisons - test_eq_count)
        survived["arithmetic_operators"] = arithmetic // 2  # Estimate
        survived["constant_values"] = constants // 2  # Estimate
        survived["logical_operators"] = logical
    except:
        # Default values if analysis fails
        survived = {
            "comparison_operators": 1,
            "arithmetic_operators": 1,
            "constant_values": 1,
            "logical_operators": 1
        }

    return survived

In [ ]:
# Assertion Guidance and Context Formatting Functions

def generate_assertion_guidance(function_analysis, original_assertions, mutation_results):
    """Generates guidance for assertion improvement based on function analysis and mutation results."""
    guidance = ["Based on the function analysis, consider the following improvements:"]

    # Check if we have edge cases covered
    if function_analysis.get("edge_cases"):
        guidance.append("- Add assertions for these edge cases:")
        for case in function_analysis.get("edge_cases"):
            guidance.append(f"  * {case}")

    # Provide type-specific guidance
    return_type = function_analysis.get("return_type")
    if return_type:
        if return_type == "list":
            guidance.append("- For list returns, add assertions checking:")
            guidance.append("  * List length")
            guidance.append("  * Expected elements")
            guidance.append("  * Element types")
            guidance.append("  * Element ordering (if relevant)")
        elif return_type == "dict":
            guidance.append("- For dictionary returns, add assertions checking:")
            guidance.append("  * Dictionary keys presence")
            guidance.append("  * Value types for each key")
            guidance.append("  * Complete structure verification")
        elif return_type == "str":
            guidance.append("- For string returns, add assertions checking:")
            guidance.append("  * Expected string value")
            guidance.append("  * String length")
            guidance.append("  * String content patterns (using regular expressions)")
        elif return_type == "bool":
            guidance.append("- For boolean returns, add assertions checking:")
            guidance.append("  * True/False for various input combinations")
            guidance.append("  * Boundary conditions that might affect the result")
        elif return_type == "numeric":
            guidance.append("- For numeric returns, add assertions checking:")
            guidance.append("  * Exact expected values")
            guidance.append("  * Range constraints (min/max)")
            guidance.append("  * Type verification (int vs float)")

    # Add guidance for complex logic
    if function_analysis.get("complex_logic"):
        guidance.append("- Function has complex logic, add assertions for:")
        guidance.append("  * Each conditional branch")
        guidance.append("  * Loop boundary cases (empty, single item, multiple items)")
        guidance.append("  * All possible return paths")

    # Add guidance based on mutation results
    if mutation_results and mutation_results.get("survived_mutations"):
        guidance.append("- Based on mutation testing, strengthen assertions for:")
        survived_mutations = mutation_results.get("survived_mutations", {})

        # Parse the survived mutations which may be a string representation of a dict
        if isinstance(survived_mutations, str):
            try:
                survived_mutations = eval(survived_mutations)
            except:
                survived_mutations = {}

        for mutation_type, count in survived_mutations.items():
            if "comparison" in mutation_type.lower():
                guidance.append("  * More precise comparisons (check exact values, not just existence)")
            if "constant" in mutation_type.lower() or "boundary" in mutation_type.lower():
                guidance.append("  * Boundary values and exact constant values")
            if "arithmetic" in mutation_type.lower():
                guidance.append("  * Calculated values with precise expected results")
            if "logical" in mutation_type.lower():
                guidance.append("  * Logical conditions (and/or operators)")

    # Check for any missing assertion types
    assertion_types = {a.get("type") for a in original_assertions}
    if "type_check" not in assertion_types:
        guidance.append("- Add type verification assertions using isinstance()")
    if "length_check" not in assertion_types and return_type in ["list", "dict", "str"]:
        guidance.append("- Add assertions checking the length of returned collections")

    return "\n".join(guidance)

def format_assertion_context(assertions):
    """Formats the assertion context for the prompt."""
    if not assertions:
        return "No assertions found in the original test code."

    context = f"Found {len(assertions)} assertions in the original test code:\n"
    for i, assertion in enumerate(assertions):
        context += f"  {i+1}. Line {assertion.get('line')} ({assertion.get('type')} assertion): assert {assertion.get('text')}"
        if assertion.get("containing_function"):
            context += f" in function {assertion.get('containing_function')}"
        context += "\n"

    return context

In [ ]:
# Enhanced Prompt Template for LLM

enhanced_prompt_template = """You are an expert Python test engineer specializing in creating robust, high-quality unit tests.

FUNCTION TO TEST:
python
{function_code}


ORIGINAL TEST CODE WITH ISSUES:
python
{original_test_code}


ASSERTION ANALYSIS:
{assertion_context}

MUTATION TESTING RESULTS:
{mutation_analysis}

GUIDANCE FOR IMPROVEMENT:
{assertion_guidance}

Your task is to rewrite the test code to maximize assertion accuracy and test coverage. Follow these specific guidelines:

IMPLEMENT PATTERN-BASED REFINEMENT:
- Replace weak assertions like `assert some_function() is not None` with stronger assertions.
- Add boundary checks for numeric assertions (e.g., test min/max values, zero, negative values).
- Combine type checks with value checks (e.g., `assert isinstance(result, list) and len(result) > 0`).
- Convert simple checks to more comprehensive validations.

IMPLEMENT SEMANTIC REFINEMENT:
- Add assertions that verify the function's core purpose.
- Create assertions for each logical path through the function.
- Ensure all edge cases identified in the guidance are tested.
- Include assertions that verify both positive and negative test cases.

BEST PRACTICES FOR ASSERTIONS:
- Add clear error messages to assertions: `assert condition, "Helpful error message"`.
- Use specific, focused assertions rather than complex compound assertions.
- Test both the structure and content of returned collections.
- For collections, check types, lengths, and specific elements.
- For functions with side effects, verify the side effect occurred properly.

MUTATION TESTING COVERAGE:
- Add assertions specifically designed to detect the survived mutations mentioned.
- Use precise equality checks rather than existence checks.
- Verify exact values, not just approximate results.
- Include assertions that would detect off-by-one errors.

Ensure the corrected test code:
- Is syntactically correct and runnable.
- Maintains the same test function structure (don't rename test functions).
- Includes significantly stronger assertions.
- Has clear, readable code with appropriate comments.

RETURN ONLY THE IMPROVED TEST CODE, with no explanations, wrapped in python and markers.

# Your improved test code here
python
# [Your improved test code will be generated here by the LLM]

"""


In [ ]:
# Advanced Assertion Correction and LLM-based Test Correction Functions

# Ensure that extract_assertions_with_context, analyze_function_for_testability,
# generate_assertion_guidance, format_assertion_context, and enhanced_prompt_template
# from previous cells are defined and available in the Colab environment before running this cell.

def advanced_assertion_correction(llm_pipeline, function_code, original_test_code,
                               mutation_results, enhanced_prompt_template, max_new_tokens=1024):
    """
    Advanced assertion correction technique using LLMs with specialized prompts.
    Implements the hybrid approach combining pattern-based and semantic refinement.

    Args:
        llm_pipeline: The LLM pipeline to use for correction.
        function_code: The function code being tested.
        original_test_code: The original tests with weak assertions.
        mutation_results: Information about mutation testing results.
        enhanced_prompt_template: The specialized prompt template.
        max_new_tokens: Maximum tokens for generation.

    Returns:
        Corrected test code with improved assertions, or None if an error occurs.
    """
    # Extract and analyze the original assertions
    # Note: extract_assertions_with_context should be defined from the user's provided code
    original_assertions = extract_assertions_with_context(original_test_code)
    assertion_analysis = {
        "total": len(original_assertions),
        "by_type": {}
    }

    # Group assertions by type for analysis
    for assertion in original_assertions:
        assertion_type = assertion.get("type", "unknown") # Use .get for safety
        if assertion_type not in assertion_analysis["by_type"]:
            assertion_analysis["by_type"][assertion_type] = []
        assertion_analysis["by_type"][assertion_type].append(assertion)

    # Analyze function code to understand what should be tested
    # Note: analyze_function_for_testability should be defined from the user's provided code
    function_analysis = analyze_function_for_testability(function_code)

    # Generate assertion guidance based on function analysis and current assertions
    # Note: generate_assertion_guidance is in next_code_cell_1.py
    assertion_guidance = generate_assertion_guidance(function_analysis, original_assertions, mutation_results)

    # Format assertion context for the prompt
    # Note: format_assertion_context is in next_code_cell_1.py
    assertion_context = format_assertion_context(original_assertions)

    # Create a mutation analysis summary if provided
    mutation_analysis_summary = ""
    if mutation_results:
        killed_ratio = f"{mutation_results.get('mutants_killed', 0)}/{mutation_results.get('total_mutants_generated', 0)}"
        mutation_analysis_summary = f"Mutation testing results: {killed_ratio} mutants killed, score: {mutation_results.get('mutation_score', 0):.2f}%.\n"

        if mutation_results.get('survived_mutations'):
            mutation_analysis_summary += "Types of mutations that survived original tests:\n"
            for mutation_type, count in mutation_results.get('survived_mutations', {}).items():
                mutation_analysis_summary += f"- {mutation_type}: {count} instances\n"
    else:
        mutation_analysis_summary = "No mutation analysis results provided."

    # Complete the specialized prompt with all context information
    # Note: enhanced_prompt_template is in next_code_cell_2.py
    full_prompt = enhanced_prompt_template.format(
        function_code=function_code,
        original_test_code=original_test_code,
        assertion_context=assertion_context,
        mutation_analysis=mutation_analysis_summary, # Use the generated summary
        assertion_guidance=assertion_guidance
    )

    try:
        model_identifier = getattr(getattr(llm_pipeline, "model", {}), "config", {}).get("_name_or_path", "Correction LLM")
        print(f"Using advanced assertion correction with {model_identifier}...")

        # Generate the corrected test code
        sequences = llm_pipeline(
            full_prompt,
            max_new_tokens=max_new_tokens,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.6,  # Balanced for creativity but precision
            top_k=40,
            top_p=0.95,
            pad_token_id=llm_pipeline.tokenizer.eos_token_id
        )
        corrected_code_suggestion = sequences[0]["generated_text"]

        # Extract the Python code block
        code_start_marker = "python\n"
        code_end_marker = "\n"
        extracted_correction = ""

        # Find the last occurrence of the code block markers
        start_index = corrected_code_suggestion.rfind(code_start_marker)
        if start_index != -1:
            start_index += len(code_start_marker)
            end_index = corrected_code_suggestion.find(code_end_marker, start_index)
            if end_index != -1:
                extracted_correction = corrected_code_suggestion[start_index:end_index]
            else: # If end marker not found after the last start marker, take rest of string
                extracted_correction = corrected_code_suggestion[start_index:]
        else:
            # Fallback: if no python block, assume the generation after prompt is the code
            # This might happen if the LLM doesn't strictly follow the marker instruction
            prompt_end_index = len(full_prompt)
            if len(corrected_code_suggestion) > prompt_end_index:
                extracted_correction = corrected_code_suggestion[prompt_end_index:].strip()
            else:
                # If the suggestion is not longer than the prompt, it likely failed or is empty
                extracted_correction = corrected_code_suggestion.strip()

        final_correction = extracted_correction.strip()

        if not final_correction:
            print("Warning: LLM did not return a code suggestion or the extraction failed.")
            return original_test_code # Return original if correction fails

        # Perform a quick analysis of the corrections made
        # Note: extract_assertions_with_context should be defined from the user's provided code
        corrected_assertions = extract_assertions_with_context(final_correction)
        print(f"Original test had {len(original_assertions)} assertions, corrected test has {len(corrected_assertions)} assertions.")

        return final_correction

    except Exception as e:
        print(f"Error during advanced assertion correction: {e}")
        return original_test_code # Return original in case of error

def correct_test_code_with_llm(correction_llm_pipeline, function_code, original_test_code, context_data, prompt_template):
    """Use LLM to correct the test code based on enhanced prompt with detailed context."""

    # Format the prompt with all the context data
    # Note: prompt_template here refers to enhanced_prompt_template from next_code_cell_2.py
    formatted_prompt = prompt_template.format(
        function_code=function_code,
        original_test_code=original_test_code,
        assertion_context=context_data.get("assertion_context", "No specific issues identified."),
        mutation_analysis=context_data.get("mutation_analysis", "No mutation analysis available."),
        assertion_guidance=context_data.get("assertion_guidance", "Improve assertion quality and coverage.")
    )

    try:
        model_name = getattr(getattr(correction_llm_pipeline, "model", {}), "config", {}).get("_name_or_path", "Correction LLM")
        print(f"Requesting corrected test code from {model_name}...")

        # Generate the corrected test code
        response = correction_llm_pipeline(
            formatted_prompt,
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.5,
            top_p=0.95,
            num_return_sequences=1,
            pad_token_id=correction_llm_pipeline.tokenizer.eos_token_id # Added pad_token_id
        )

        generated_text = response[0]["generated_text"]

        # Extract the Python code from the response
        corrected_code = ""
        # Using the same robust extraction logic as in advanced_assertion_correction
        code_start_marker = "python\n"
        code_end_marker = "\n"

        start_index = generated_text.rfind(code_start_marker)
        if start_index != -1:
            start_index += len(code_start_marker)
            end_index = generated_text.find(code_end_marker, start_index)
            if end_index != -1:
                corrected_code = generated_text[start_index:end_index]
            else:
                corrected_code = generated_text[start_index:]
        else:
            prompt_end_index = len(formatted_prompt)
            if len(generated_text) > prompt_end_index:
                corrected_code = generated_text[prompt_end_index:].strip()
            else:
                corrected_code = generated_text.strip()

        final_corrected_code = corrected_code.strip()

        if not final_corrected_code:
            print("Warning: Could not extract corrected code from LLM response. Returning original.")
            return original_test_code # Return original if extraction fails

        print(f"Successfully generated corrected test code ({len(final_corrected_code.splitlines())} lines).")
        return final_corrected_code

    except Exception as e:
        print(f"Error generating corrected test code: {str(e)}")
        return original_test_code # Return original in case of error

In [ ]:
# --- Mutation Operators ---
def mutate_comparison_operator(node):
    """Mutates comparison operators (e.g., > to >=, == to !=)."""
    if isinstance(node, ast.Compare):
        original_op = type(node.ops[0])
        mutated = False
        if original_op == ast.Gt: node.ops[0] = ast.LtE(); mutated = True
        elif original_op == ast.Lt: node.ops[0] = ast.GtE(); mutated = True
        elif original_op == ast.GtE: node.ops[0] = ast.Lt(); mutated = True
        elif original_op == ast.LtE: node.ops[0] = ast.Gt(); mutated = True
        elif original_op == ast.Eq: node.ops[0] = ast.NotEq(); mutated = True
        elif original_op == ast.NotEq: node.ops[0] = ast.Eq(); mutated = True
        elif original_op == ast.Is: node.ops[0] = ast.IsNot(); mutated = True
        elif original_op == ast.IsNot: node.ops[0] = ast.Is(); mutated = True
        return mutated
    return False

def mutate_arithmetic_operator(node):
    """Mutates arithmetic operators (e.g., + to -, * to /)."""
    if isinstance(node, ast.BinOp):
        original_op = type(node.op)
        mutated = False
        if original_op == ast.Add: node.op = ast.Sub(); mutated = True
        elif original_op == ast.Sub: node.op = ast.Add(); mutated = True
        elif original_op == ast.Mult: node.op = ast.Div(); mutated = True
        elif original_op == ast.Div: node.op = ast.Mult(); mutated = True
        return mutated
    return False

def mutate_constant_value(node):
    """Mutates constant numeric values (e.g., 1 to 0, 0 to 1, N to N+1)."""
    if isinstance(node, ast.Constant) and isinstance(node.value, (int, float)):
        original_value = node.value
        mutated = False
        if original_value == 1: node.value = 0; mutated = True
        elif original_value == 0: node.value = 1; mutated = True
        else: node.value = original_value + 1; mutated = True
        return mutated
    return False

def mutate_logical_operator(node):
    """Mutates logical operators (e.g., and to or, or to and)."""
    if isinstance(node, ast.BoolOp):
        mutated = False
        if isinstance(node.op, ast.And): node.op = ast.Or(); mutated = True
        elif isinstance(node.op, ast.Or): node.op = ast.And(); mutated = True
        return mutated
    return False

def evaluate_corrected_test_code(function_id, function_code, entry_point, corrected_test_code, correction_llm_name):
    """Evaluates the corrected test code."""
    evaluation = {
        "unique_id": function_id,
        "correction_llm_model": correction_llm_name,
        "corrected_test_code": corrected_test_code,
        "corrected_syntactic_validity": False,
        "corrected_extracted_assertions": [],
        "corrected_num_assertions": 0,
        "corrected_execution_passes": None,
        "corrected_execution_error": None
    }

    if not corrected_test_code or not isinstance(corrected_test_code, str) or not corrected_test_code.strip():
        evaluation["corrected_execution_error"] = "Empty or invalid corrected test code."
        return evaluation

    try:
        ast.parse(corrected_test_code)
        evaluation["corrected_syntactic_validity"] = True
    except SyntaxError as e:
        evaluation["corrected_execution_error"] = f"SyntaxError in corrected code: {e}"
        return evaluation

    evaluation["corrected_extracted_assertions"] = extract_assertions_with_context(corrected_test_code)
    evaluation["corrected_num_assertions"] = len(evaluation["corrected_extracted_assertions"])

    if evaluation["corrected_syntactic_validity"] and entry_point and isinstance(function_code, str) and function_code.strip():
        safe_function_id = re.sub(r"[^a-zA-Z0-9_]", "", str(function_id))
        safe_llm_name = re.sub(r"[^a-zA-Z0-9_]", "", correction_llm_name)
        temp_module_name = f"temp_corrected_test_module_{safe_function_id}_{safe_llm_name}"

        full_code_to_execute = function_code + "\n\n" + corrected_test_code

        if not re.search(r"if __name__ == .__main__.:", corrected_test_code, re.IGNORECASE) and "pytest" not in corrected_test_code.lower():
            test_function_calls = []
            try:
                parsed_tests = ast.parse(corrected_test_code)
                for node in parsed_tests.body:
                    if isinstance(node, ast.FunctionDef) and node.name.startswith("test"):
                        test_function_calls.append(f"{node.name}()")
                if test_function_calls:
                    full_code_to_execute += "\n\n# Auto-added test runner for corrected tests\n" + "\n".join(test_function_calls)
            except SyntaxError:
                pass

        try:
            with open(f"{temp_module_name}.py", "w", encoding="utf-8") as f:
                f.write(full_code_to_execute)

            import subprocess
            process = subprocess.run(["python", f"{temp_module_name}.py"], capture_output=True, text=True, timeout=20, encoding="utf-8", errors="replace")

            if process.returncode == 0:
                evaluation["corrected_execution_passes"] = True
            else:
                evaluation["corrected_execution_passes"] = False
                evaluation["corrected_execution_error"] = process.stderr[:1000]
        except Exception as e:
            evaluation["corrected_execution_error"] = f"Runtime error: {str(e)[:500]}"
            evaluation["corrected_execution_passes"] = False
        finally:
            if os.path.exists(f"{temp_module_name}.py"):
                os.remove(f"{temp_module_name}.py")

    return evaluation

def generate_mutants(original_code_str, num_mutants_per_function=5):
    mutants = []
    if not original_code_str or not isinstance(original_code_str, str) or not original_code_str.strip():
        return mutants
    try:
        original_tree = ast.parse(original_code_str)
    except SyntaxError:
        return mutants

    generated_mutant_codes = set()
    for _ in range(num_mutants_per_function * 10):
        if len(mutants) >= num_mutants_per_function:
            break

        mutant_tree = ast.parse(original_code_str)
        nodes_in_mutant_tree = [n for n in ast.walk(mutant_tree)]
        random.shuffle(nodes_in_mutant_tree)

        applied_mutation = False
        for node_to_mutate in nodes_in_mutant_tree:
            operators = [mutate_comparison_operator, mutate_arithmetic_operator, mutate_logical_operator,
                         mutate_constant_value]
            shuffled_operators = random.sample(operators, len(operators))
            for operator_func in shuffled_operators:
                try:
                    if operator_func(node_to_mutate):
                        applied_mutation = True
                        break
                except:
                    continue
            if applied_mutation:
                break

        if applied_mutation:
            try:
                mutant_code = ast.unparse(mutant_tree)
                if mutant_code != original_code_str and mutant_code not in generated_mutant_codes:
                    ast.parse(mutant_code)  # Verify syntax
                    mutants.append(mutant_code)
                    generated_mutant_codes.add(mutant_code)
            except:
                pass
    return mutants[:num_mutants_per_function]

def run_tests_on_mutant(function_entry_point, mutant_code_str, test_code_str, original_function_code_str):
    if not test_code_str or not isinstance(test_code_str, str) or not test_code_str.strip() or \
       not mutant_code_str or not isinstance(mutant_code_str, str) or not mutant_code_str.strip():
        return False

    full_script_content = mutant_code_str + "\n\n" + test_code_str

    if not re.search(r"if __name__ == .__main__.:", test_code_str, re.IGNORECASE) and "pytest" not in test_code_str.lower():
        test_function_calls = []
        try:
            parsed_tests = ast.parse(test_code_str)
            for node in parsed_tests.body:
                if isinstance(node, ast.FunctionDef) and node.name.startswith("test"):
                    test_function_calls.append(f"{node.name}()")
            if test_function_calls:
                full_script_content += "\n\n# Auto-added test runner\n" + "\n".join(test_function_calls)
        except SyntaxError:
            pass

    safe_entry_point = re.sub(r"[^a-zA-Z0-9_]", "", str(function_entry_point))
    temp_script_path = f"temp_mutant_test_runner_{safe_entry_point}.py"

    try:
        with open(temp_script_path, "w", encoding="utf-8") as f:
            f.write(full_script_content)

        process = subprocess.run([sys.executable, temp_script_path], capture_output=True, text=True, timeout=20)
        return process.returncode != 0  # Killed if tests fail
    except:
        return False
    finally:
        if os.path.exists(temp_script_path):
            os.remove(temp_script_path)

In [ ]:
def evaluate_assertion_correction_effectiveness(BASE_DRIVE_PATH):
    """Evaluate the effectiveness of assertion corrections by re-running mutation testing."""
    print("\n--- Evaluating Assertion Correction Effectiveness ---")

    correction_summary_path = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results", "advanced_correction_summary.csv")
    if not os.path.exists(correction_summary_path):
        print(f"Correction summary not found at {correction_summary_path}. Run assertion correction first.")
        return

    try:
        correction_df = pd.read_csv(correction_summary_path)
        print(f"Loaded {len(correction_df)} corrected test cases for evaluation.")
    except Exception as e:
        print(f"Error loading correction summary: {e}")
        return

    # Filter to only include syntactically valid corrections
    valid_corrections = correction_df[correction_df["corrected_syntactic_validity"] == True].copy()
    print(f"Found {len(valid_corrections)} syntactically valid corrections to evaluate.")

    if valid_corrections.empty:
        print("No valid corrections to evaluate.")
        return

    # For each corrected test, re-run mutation testing
    evaluation_results = []

    for index, row in valid_corrections.iterrows():
        unique_id = row["unique_id"]
        function_code = str(row["function_code"])
        corrected_test_code = str(row["corrected_test_code"])
        entry_point = row.get("entry_point", "")
        original_mutation_score = row.get("original_mutation_score", 0)

        if pd.isna(function_code) or pd.isna(corrected_test_code) or pd.isna(entry_point):
            print(f"Skipping {unique_id} due to missing data.")
            continue

        print(f"\nRe-evaluating function {unique_id} with corrected assertions...")

        # Generate identical mutants as used in the original evaluation
        num_mutants = 7  # Use same number for fair comparison
        try:
            mutants = generate_mutants(function_code, num_mutants_per_function=num_mutants)
        except:
            print(f"Error generating mutants for {unique_id}, skipping.")
            continue

        if not mutants:
            print(f"No mutants generated for {unique_id}, skipping evaluation.")
            continue

        # Test each mutant with the corrected test code
        mutants_killed = 0
        for i, mutant_code in enumerate(mutants):
            try:
                killed = run_tests_on_mutant(entry_point, mutant_code, corrected_test_code, function_code)
                if killed:
                    mutants_killed += 1
            except:
                print(f"Error testing mutant {i} for {unique_id}")

        # Calculate the new mutation score
        new_mutation_score = (mutants_killed / len(mutants)) * 100 if mutants else 0

        # Calculate improvement
        if pd.isna(original_mutation_score):
            improvement = "Unknown (original score not available)"
            percent_improvement = 0
        else:
            improvement = new_mutation_score - original_mutation_score
            percent_improvement = (improvement / original_mutation_score * 100) if original_mutation_score > 0 else 0

        print(f"Original score: {original_mutation_score:.2f}%, New score: {new_mutation_score:.2f}%, Improvement: {improvement:.2f} points")

        evaluation_results.append({
            "unique_id": unique_id,
            "original_mutation_score": original_mutation_score,
            "corrected_mutation_score": new_mutation_score,
            "absolute_improvement": improvement,
            "percent_improvement": percent_improvement,
            "original_assertions": row.get("original_num_assertions"),
            "corrected_assertions": row.get("corrected_num_assertions"),
            "mutants_tested": len(mutants),
            "mutants_killed": mutants_killed
        })

    # Save and display results
    if evaluation_results:
        eval_df = pd.DataFrame(evaluation_results)
        output_path = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results", "correction_effectiveness.csv")
        eval_df.to_csv(output_path, index=False)

        print("\n--- Assertion Correction Effectiveness Summary ---")
        print(f"Average original mutation score: {eval_df['original_mutation_score'].mean():.2f}%")
        print(f"Average corrected mutation score: {eval_df['corrected_mutation_score'].mean():.2f}%")
        print(f"Average absolute improvement: {eval_df['absolute_improvement'].mean():.2f} percentage points")
        print(f"Average percent improvement: {eval_df['percent_improvement'].mean():.2f}%")
        print(f"Corrections with positive improvement: {len(eval_df[eval_df['absolute_improvement'] > 0])}/{len(eval_df)} ({len(eval_df[eval_df['absolute_improvement'] > 0])/len(eval_df)*100:.2f}%)")

        display(eval_df.head())

        # Create summary by original LLM
        if 'original_llm_model' in correction_df.columns:
            merged_eval = pd.merge(eval_df, correction_df[['unique_id', 'original_llm_model']], on='unique_id', how='left')
            model_summary = merged_eval.groupby('original_llm_model').agg({
                'original_mutation_score': 'mean',
                'corrected_mutation_score': 'mean',
                'absolute_improvement': 'mean'
            }).round(2).reset_index()

            print("\nImprovement by original LLM model:")
            display(model_summary)
    else:
        print("No mutation testing results to report.")

    print("\n--- Assertion Correction Effectiveness Evaluation Completed ---")

In [ ]:
def run_advanced_assertion_correction_pipeline(BASE_DRIVE_PATH, correction_llm_pipeline=None):
    """
    Implements advanced assertion correction pipeline using specialized LLM prompts,
    pattern-based refinement, and semantic refinement as described in the methodology.
    """
    print("\n--- Starting Advanced Assertion Correction Pipeline ---")

    mutation_results_path = os.path.join(BASE_DRIVE_PATH, "mutation_testing_results", "mutation_testing_summary.csv")
    llm_tests_path = os.path.join(BASE_DRIVE_PATH, "llm_generated_tests", "llm_generated_tests_output.csv")
    baseline_eval_path = os.path.join(BASE_DRIVE_PATH, "llm_generated_tests", "baseline_evaluation_summary.csv")

    if not os.path.exists(mutation_results_path) or not os.path.exists(llm_tests_path) or not os.path.exists(baseline_eval_path):
        print("Required input files (mutation results, LLM tests, or baseline eval) not found. Run previous steps.")
        return

    try:
        mutation_df = pd.read_csv(mutation_results_path)
        llm_tests_df = pd.read_csv(llm_tests_path)
        baseline_df = pd.read_csv(baseline_eval_path)

        # Merge dataframes to get comprehensive view of each test case
        merged_df = pd.merge(llm_tests_df, baseline_df[["unique_id", "llm_model", "execution_passes", "syntactic_validity", "num_assertions"]],
                             on=["unique_id", "llm_model"], how="left", suffixes=("", "_baseline"))
        merged_df = pd.merge(merged_df, mutation_df[["unique_id", "llm_model", "mutation_score", "mutants_killed", "total_mutants_generated"]],
                             on=["unique_id", "llm_model"], how="left", suffixes=("", "_mutation"))
        print(f"Loaded and merged all relevant data: {len(merged_df)} entries.")
    except Exception as e:
        print(f"Error loading or merging data: {e}")
        return

    # Use comprehensive criteria for selecting candidates for correction
    correction_threshold = 75.0
    candidates_for_correction = merged_df[
        # Either low mutation score
        (merged_df["mutation_score"].fillna(101) < correction_threshold) |
        # Or execution failures despite valid syntax
        ((merged_df.get("execution_passes_baseline") == False) & (merged_df.get("syntactic_validity_baseline") == True)) |
        # Or very few assertions
        (merged_df.get("num_assertions", pd.Series([5] * len(merged_df))) < 3)
    ].copy()

    print(f"Identified {len(candidates_for_correction)} test suites as candidates for correction.")
    if candidates_for_correction.empty:
        print("No candidates for assertion correction based on current criteria.")
        return

    if correction_llm_pipeline is None:
        print("Correction LLM pipeline not provided, loading DeepSeek-Coder-1.3B-instruct with quantization.")
        # Implement a function to load LLM here
        # For example:
        # correction_llm_pipeline = load_correction_llm()
        if not correction_llm_pipeline:
            print("Failed to load LLM for correction. Aborting.")
            return
    else:
        model_name = correction_llm_pipeline.model.config._name_or_path if hasattr(correction_llm_pipeline.model, "config") else "Passed LLM"
        print(f"Using provided LLM for correction: {model_name}")

    correction_results_list = []

    # Process candidates for correction
    for index, row in candidates_for_correction.iterrows():
        unique_id = row["unique_id"]
        function_code = str(row["function_code"])
        original_test_code = str(row["generated_test_code"])
        llm_model_source = row["llm_model"]
        entry_point = str(row.get("entry_point", ""))

        # Prepare mutation results data structure for the correction process
        mutation_results_data = {
            "mutation_score": row.get("mutation_score"),
            "mutants_killed": row.get("mutants_killed", 0),
            "total_mutants_generated": row.get("total_mutants_generated", 0),
            "survived_mutations": analyze_survived_mutations(function_code, original_test_code)
        }

        print(f"\nRunning advanced assertion correction for function {unique_id} (original tests by {llm_model_source})...")

        # Use advanced assertion correction function
        corrected_test_code = advanced_assertion_correction(
            correction_llm_pipeline,
            function_code,
            original_test_code,
            mutation_results_data,
            enhanced_prompt_template
        )

        # Evaluate the corrected test code
        if corrected_test_code:
            corrected_evaluation = evaluate_corrected_test_code(
                unique_id, function_code, entry_point, corrected_test_code,
                correction_llm_pipeline.model.config._name_or_path
            )

            # Store the results
            correction_results_list.append({
                "unique_id": unique_id,
                "original_llm_model": llm_model_source,
                "correction_llm_model": correction_llm_pipeline.model.config._name_or_path,
                "function_code": function_code,
                "original_test_code": original_test_code,
                "corrected_test_code": corrected_test_code,
                "original_mutation_score": row.get("mutation_score"),
                "original_num_assertions": row.get("num_assertions"),
                "corrected_num_assertions": corrected_evaluation.get("corrected_num_assertions"),
                "corrected_syntactic_validity": corrected_evaluation.get("corrected_syntactic_validity"),
                "corrected_execution_passes": corrected_evaluation.get("corrected_execution_passes"),
                "corrected_execution_error": corrected_evaluation.get("corrected_execution_error")
            })
        else:
            print(f"Correction failed for {unique_id}")

        # Small delay between operations
        time.sleep(0.5)

    if correction_results_list:
        # Save the correction results
        correction_results_df = pd.DataFrame(correction_results_list)
        output_dir = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results")
        os.makedirs(output_dir, exist_ok=True)
        correction_results_path = os.path.join(output_dir, "advanced_correction_summary.csv")
        correction_results_df.to_csv(correction_results_path, index=False)

        print(f"\nAdvanced assertion correction results saved to {correction_results_path}")
        print("\nSummary of corrections:")
        print(f"- Total corrections attempted: {len(correction_results_df)}")
        print(f"- Syntactically valid: {correction_results_df['corrected_syntactic_validity'].sum()}")
        if 'corrected_execution_passes' in correction_results_df.columns:
            passes = correction_results_df['corrected_execution_passes'].sum()
            print(f"- Execution passes: {passes} ({passes/len(correction_results_df)*100:.1f}%)")
        print(f"- Average original assertions: {correction_results_df['original_num_assertions'].mean():.2f}")
        print(f"- Average corrected assertions: {correction_results_df['corrected_num_assertions'].mean():.2f}")

        if not correction_results_df.empty:
            display(correction_results_df[['unique_id', 'original_llm_model', 'original_num_assertions',
                                        'corrected_num_assertions', 'corrected_syntactic_validity',
                                        'corrected_execution_passes']].head())
    else:
        print("No successful corrections to save.")

    print("\n--- Advanced Assertion Correction Pipeline Completed ---")

In [ ]:

# Define base drive path where data is stored
BASE_DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Assertion-Accuracy-Research/Data"

# Install required packages
!pip install -q transformers peft datasets evaluate rouge_score

# Import common libraries
import os
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType, PeftConfig, PeftModel
from transformers import BitsAndBytesConfig
from sklearn.model_selection import train_test_split
import evaluate
import warnings
warnings.filterwarnings("ignore")

# Create necessary directories
os.makedirs(BASE_DRIVE_PATH, exist_ok=True)
os.makedirs(os.path.join(BASE_DRIVE_PATH, "assertion_correction_results"), exist_ok=True)

print("Setup complete. Base path:", BASE_DRIVE_PATH)


Setup complete. Base path: /content/drive/MyDrive/Colab Notebooks/Assertion-Accuracy-Research/Data


In [ ]:
#  Dataset Preparation Function

def prepare_assertion_correction_dataset(BASE_DRIVE_PATH, tokenizer, max_length=1024):
  """
  Prepare dataset for fine-tuning assertion correction models using existing corrections.

  Args:
    BASE_DRIVE_PATH: Base path where correction data is stored
    tokenizer: The tokenizer for the model to be fine-tuned
    max_length: Maximum sequence length for tokenization

  Returns:
    train_dataset, eval_dataset: PyTorch datasets ready for fine-tuning
  """
  print("\n--- Preparing Assertion Correction Dataset for Fine-tuning ---")

  # Check for correction data
  correction_summary_path = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results", "advanced_correction_summary.csv")

  # If the file doesn't exist, create a sample dataset for demonstration
  if not os.path.exists(correction_summary_path):
    print(f"Correction summary not found at {correction_summary_path}")
    print("Creating sample dataset for demonstration...")

    # Create directory if it doesn't exist
    os.makedirs(os.path.join(BASE_DRIVE_PATH, "assertion_correction_results"), exist_ok=True)

    # Create sample data
    sample_data = []
    for i in range(20):  # Create 20 sample entries
      sample_data.append({
        "function_id": f"sample_func_{i}",
        "function_code": f"def sample_function_{i}(x, y):\n    return x + y",
        "original_test_code": f"def test_sample_{i}():\n    assert sample_function_{i}(1, 2) == 3",
        "corrected_test_code": f"def test_sample_{i}():\n    assert sample_function_{i}(1, 2) == 3\n    assert sample_function_{i}(0, 0) == 0\n    assert sample_function_{i}(-1, 1) == 0",
        "original_num_assertions": 1,
        "corrected_num_assertions": 3,
        "corrected_syntactic_validity": True,
        "corrected_execution_passes": True,
        "model_name": "sample_model"
      })

    # Save sample data
    pd.DataFrame(sample_data).to_csv(correction_summary_path, index=False)
    print(f"Created sample dataset with {len(sample_data)} entries at {correction_summary_path}")

  # Load correction data
  try:
    df = pd.read_csv(correction_summary_path)
    print(f"Loaded {len(df)} correction examples")
  except Exception as e:
    print(f"Error loading correction data: {e}")
    return None, None

  # Fix: Ensure all required columns exist, with default values if missing
  required_columns = [
    "function_code", "original_test_code", "corrected_test_code",
    "original_num_assertions", "corrected_num_assertions",
    "corrected_syntactic_validity", "corrected_execution_passes"
  ]

  for col in required_columns:
    if col not in df.columns:
      if col in ["corrected_syntactic_validity", "corrected_execution_passes"]:
        df[col] = True  # Default to True for boolean flags
      elif col in ["original_num_assertions", "corrected_num_assertions"]:
        df[col] = 1  # Default to 1 for assertion counts
      else:
        df[col] = ""  # Default to empty string for code fields

  # Fix: More lenient filtering to ensure we get enough training examples
  # Only require that corrected code has valid syntax and passes execution
  successful_df = df[
      (df["corrected_syntactic_validity"] == True) &
      (df["corrected_execution_passes"] == True)
  ].copy()

  # If still not enough examples, create synthetic examples
  if len(successful_df) < 10:
    print(f"Found only {len(successful_df)} successful corrections, adding synthetic examples")

    # Create synthetic examples
    synthetic_examples = []
    for i in range(max(10 - len(successful_df), 0)):
      synthetic_examples.append({
        "function_code": f"def synthetic_func_{i}(a, b):\n    return a * b",
        "original_test_code": f"def test_synthetic_{i}():\n    assert synthetic_func_{i}(2, 3) == 6",
        "corrected_test_code": f"def test_synthetic_{i}():\n    assert synthetic_func_{i}(2, 3) == 6\n    assert synthetic_func_{i}(0, 5) == 0\n    assert synthetic_func_{i}(-1, -1) == 1",
        "original_num_assertions": 1,
        "corrected_num_assertions": 3,
        "corrected_syntactic_validity": True,
        "corrected_execution_passes": True
      })

    # Add synthetic examples to successful dataframe
    synthetic_df = pd.DataFrame(synthetic_examples)
    successful_df = pd.concat([successful_df, synthetic_df], ignore_index=True)

  print(f"Using {len(successful_df)} examples for training")

  # Prepare instruction format for each example
  training_examples = []
  for idx, row in successful_df.iterrows():
    # Format input as instruction
    instruction = f"""Function to test:
```python
{row['function_code']}
```

Original test code with weak assertions:
```python
{row['original_test_code']}
```

Improve the assertions in this test code to be more comprehensive and accurate."""

    # The expected output is the corrected test code
    response = row['corrected_test_code']

    # Create a training example
    training_examples.append({
        "instruction": instruction,
        "input": "", # Input can be empty as instruction contains everything
        "output": response
    })

  # Split into training and validation sets
  train_examples, eval_examples = train_test_split(training_examples, test_size=0.2, random_state=42)

  print(f"Created {len(train_examples)} training examples and {len(eval_examples)} validation examples")

  # Convert to HF datasets
  train_dataset = Dataset.from_pandas(pd.DataFrame(train_examples))
  eval_dataset = Dataset.from_pandas(pd.DataFrame(eval_examples))

  # Tokenize the datasets
  def tokenize_function(examples):
    # Combine instruction and input for the prompt
    if examples["input"] and any(examples["input"]):
      prompts = [f"<s>[INST] {instruction} {input} [/INST]"
                 for instruction, input in zip(examples["instruction"], examples["input"])]
    else:
      prompts = [f"<s>[INST] {instruction} [/INST]"
                 for instruction in examples["instruction"]]

    tokenized_prompts = tokenizer(prompts, truncation=True, max_length=max_length // 2)

    # Tokenize the responses/outputs
    tokenized_outputs = tokenizer(examples["output"], truncation=True, max_length=max_length // 2)

    # Combine them into input_ids and attention_mask
    input_ids = []
    attention_mask = []
    labels = []

    for i in range(len(tokenized_prompts["input_ids"])):
      # Get the prompt tokens (with special tokens)
      prompt_input_ids = tokenized_prompts["input_ids"][i]
      prompt_attention_mask = tokenized_prompts["attention_mask"][i]

      # Get the output/response tokens
      output_input_ids = tokenized_outputs["input_ids"][i]

      # Create combined tokens - the prompt followed by the expected output
      combined_input_ids = prompt_input_ids + output_input_ids
      combined_attention_mask = prompt_attention_mask + ([1] * len(output_input_ids)) # Ensure attention mask covers output

      # Create labels: -100 for prompt tokens (they're ignored in loss calculation)
      # and actual token IDs for response tokens
      combined_labels = [-100] * len(prompt_input_ids) + output_input_ids

      # Ensure we don't exceed max_length
      if len(combined_input_ids) > max_length:
        combined_input_ids = combined_input_ids[:max_length]
        combined_attention_mask = combined_attention_mask[:max_length]
        combined_labels = combined_labels[:max_length]

      input_ids.append(combined_input_ids)
      attention_mask.append(combined_attention_mask)
      labels.append(combined_labels)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

  # Apply tokenization
  train_tokenized = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
  eval_tokenized = eval_dataset.map(tokenize_function, batched=True, remove_columns=eval_dataset.column_names)

  # Convert to PyTorch tensors
  train_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
  eval_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

  print("Dataset preparation complete")

  return train_tokenized, eval_tokenized


In [ ]:
#  LoRA Model Setup Function

def setup_lora_model_for_finetuning(
    base_model_name="deepseek-ai/deepseek-coder-1.3b-instruct",
    use_quantization=True,
    lora_r=8,
    lora_alpha=32,
    lora_dropout=0.05
):
  """
  Set up a model with LoRA adapters for fine-tuning.

  Args:
    base_model_name: Name or path of the base model
    use_quantization: Whether to use 4-bit quantization for memory efficiency
    lora_r: LoRA attention dimension
    lora_alpha: Alpha parameter for LoRA scaling
    lora_dropout: Dropout probability for LoRA layers

  Returns:
    model: The model with LoRA adapters
    tokenizer: The model's tokenizer
  """
  print(f"\n--- Setting up {base_model_name} with LoRA adapters ---")

  # Load tokenizer
  tokenizer = AutoTokenizer.from_pretrained(base_model_name)
  tokenizer.pad_token = tokenizer.eos_token # Set pad token

  # Configure quantization if enabled
  quantization_config = None # Initialize to None
  if use_quantization:
    print("Using 4-bit quantization for memory-efficient training")
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )
  else:
    print("Using full precision for training")

  # Load the model with quantization if enabled
  try:
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=quantization_config,
        device_map="auto" # Automatically maps model to available devices (e.g., GPU)
    )
  except Exception as e:
    print(f"Error loading model: {e}")
    print("Trying alternative loading method...")
    # Alternative loading method without specific config attributes
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        load_in_4bit=use_quantization,
        device_map="auto"
    )

  # Configure LoRA
  print(f"Configuring LoRA with r={lora_r}, alpha={lora_alpha}, dropout={lora_dropout}")

  # Fix: Determine target modules based on model architecture
  # This handles different model architectures more robustly
  if "llama" in base_model_name.lower() or "deepseek" in base_model_name.lower():
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
  elif "phi" in base_model_name.lower():
    target_modules = ["q_proj", "k_proj", "v_proj", "dense"]
  else:
    # Default target modules that work for many transformer architectures
    target_modules = ["query", "key", "value", "dense"]

  lora_config = LoraConfig(
      r=lora_r,
      lora_alpha=lora_alpha,
      lora_dropout=lora_dropout,
      bias="none",
      task_type=TaskType.CAUSAL_LM,
      target_modules=target_modules
  )

  # Wrap the model with LoRA adapters
  model = get_peft_model(model, lora_config)

  # Print trainable vs total parameters
  trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  total_params = sum(p.numel() for p in model.parameters())
  print(f"Trainable parameters: {trainable_params} ({trainable_params / total_params * 100:.2f}% of total)")

  model.print_trainable_parameters() # Method to display trainable parameters summary

  return model, tokenizer


In [ ]:
# Training Loop Function

def train_assertion_correction_model(
    BASE_DRIVE_PATH,
    model,
    tokenizer,
    train_dataset,
    eval_dataset,
    output_dir_name="assertion-correction-model-v2",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    warmup_ratio=0.03
):
  """
  Fine-tune a model with LoRA adapters on assertion correction tasks.

  Args:
    BASE_DRIVE_PATH: Base directory for saving models and logs
    model: The model with LoRA adapters
    tokenizer: The model's tokenizer
    train_dataset: Training dataset
    eval_dataset: Evaluation dataset
    output_dir_name: Subdirectory name for saving model checkpoints
    num_train_epochs: Number of training epochs
    per_device_train_batch_size: Batch size per device
    gradient_accumulation_steps: Steps for gradient accumulation
    learning_rate: Learning rate for training
    warmup_ratio: Portion of training steps for learning rate warmup

  Returns:
    trainer: The trainer object after training
  """
  print("\n--- Starting Fine-tuning for Assertion Correction ---")

  # Set up output directory
  output_dir = os.path.join(BASE_DRIVE_PATH, output_dir_name)
  os.makedirs(output_dir, exist_ok=True)

  # Load metric for evaluation
  try:
    rouge_metric = evaluate.load("rouge")
  except Exception as e:
    print(f"Error loading rouge metric: {e}. Installing required packages...")
    !pip install -q evaluate rouge_score
    rouge_metric = evaluate.load("rouge")

  def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # Decode predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Decode reference labels, filtering out -100 values (used for masking prompt tokens)
    # Replace -100 with pad_token_id before decoding
    labels = [[token_id if token_id != -100 else tokenizer.pad_token_id for token_id in label_example] for label_example in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    result = rouge_metric.compute(predictions=decoded_predictions, references=decoded_labels, use_stemmer=True)

    # Log a sample prediction for inspection
    sample_idx = 0
    if len(decoded_labels) > sample_idx and len(decoded_predictions) > sample_idx:
        print("\nSample prediction comparison:")
        print(f"Reference:\n{decoded_labels[sample_idx][:200]}...")
        print(f"Prediction:\n{decoded_predictions[sample_idx][:200]}...")
    else:
        print("\nNot enough samples to display prediction comparison.")

    # Add precision, recall, fmeasure for ROUGE scores to the logs
    prediction_lens = [len(pred.split()) for pred in decoded_predictions]
    result["gen_len"] = sum(prediction_lens) / len(prediction_lens)
    return {key: value for key, value in result.items()}

  # Configure training arguments
  training_args = TrainingArguments(
      output_dir=output_dir,
      num_train_epochs=num_train_epochs,
      per_device_train_batch_size=per_device_train_batch_size,
      per_device_eval_batch_size=per_device_train_batch_size,
      gradient_accumulation_steps=gradient_accumulation_steps,
      evaluation_strategy="epoch",
      save_strategy="epoch",
      logging_dir=os.path.join(output_dir, "logs"),
      logging_steps=10,
      learning_rate=learning_rate,
      weight_decay=0.01,
      warmup_ratio=warmup_ratio,
      lr_scheduler_type="cosine",
      load_best_model_at_end=True,
      metric_for_best_model="rouge1",
      push_to_hub=False,
      report_to="tensorboard",
      remove_unused_columns=False
  )

  # Configure the trainer
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics,
      tokenizer=tokenizer
  )

  # Train the model
  print(f"Starting training for {num_train_epochs} epochs...")
  if torch.cuda.is_available():
    torch.cuda.empty_cache() # Clear GPU memory if CUDA is available

  try:
    trainer.train()
  except Exception as e:
    print(f"Error during training: {e}")
    # Try with smaller batch size if we encounter CUDA out of memory
    if "CUDA out of memory" in str(e):
      print("CUDA out of memory error. Trying with smaller batch size...")
      training_args.per_device_train_batch_size = 1
      training_args.per_device_eval_batch_size = 1
      training_args.gradient_accumulation_steps = 16
      trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          compute_metrics=compute_metrics,
          tokenizer=tokenizer
      )
      try:
        trainer.train()
      except Exception as e2:
        print(f"Still encountering error: {e2}")
        return None
    else:
      return None

  # Save the final model
  final_model_path = os.path.join(output_dir, "final_model_checkpoint")
  trainer.save_model(final_model_path)
  print(f"Training complete. Model saved to {final_model_path}")

  return trainer


In [ ]:
# Model Loading Function

def load_finetuned_assertion_model(BASE_DRIVE_PATH, model_dir_name="assertion-correction-model-v2"):
  """
  Load the fine-tuned assertion correction model with LoRA adapters.

  Args:
    BASE_DRIVE_PATH: Base directory where models are stored
    model_dir_name: Name of the model subdirectory (should match the one used in training)

  Returns:
    pipe: A text generation pipeline using the fine-tuned model, or None if loading fails.
  """
  print("\n--- Loading Fine-tuned Assertion Correction Model ---")

  # Path to the final saved model checkpoint from training
  model_path = os.path.join(BASE_DRIVE_PATH, model_dir_name, "final_model_checkpoint")

  if not os.path.exists(model_path):
    print(f"Model not found at {model_path}. Please ensure training was successful and the path is correct.")
    return None

  try:
    # Load the PEFT configuration from the saved model directory
    peft_config = PeftConfig.from_pretrained(model_path)

    print(f"Loading base model: {peft_config.base_model_name_or_path}")

    # Configure quantization (should match training configuration for consistency)
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )

    # Load base model with the same quantization used during training
    try:
      base_model = AutoModelForCausalLM.from_pretrained(
          peft_config.base_model_name_or_path,
          quantization_config=quantization_config,
          device_map="auto"
      )
    except Exception as e:
      print(f"Error loading model with quantization: {e}")
      print("Trying alternative loading method...")
      # Alternative loading method without specific config attributes
      base_model = AutoModelForCausalLM.from_pretrained(
          peft_config.base_model_name_or_path,
          load_in_4bit=True,
          device_map="auto"
      )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)
    tokenizer.pad_token = tokenizer.eos_token # Ensure pad token is set

    # Load LoRA adapters onto the base model
    print("Loading LoRA adapters...")
    model = PeftModel.from_pretrained(base_model, model_path)

    # Optional: merge LoRA weights for faster inference
    try:
      model = model.merge_and_unload()
    except Exception as e:
      print(f"Warning: Could not merge LoRA weights: {e}")
      print("Continuing with adapter model...")

    model.eval() # Set model to evaluation mode

    # Create text generation pipeline
    pipe = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=2048,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
    )

    print("Fine-tuned model loaded successfully and pipeline created.")
    return pipe

  except Exception as e:
    print(f"Error loading fine-tuned model: {e}")
    import traceback
    traceback.print_exc()
    return None


In [ ]:
# Complete Pipeline Function

def run_assertion_model_finetuning_pipeline(
    BASE_DRIVE_PATH,
    base_model_name="deepseek-ai/deepseek-coder-1.3b-instruct",
    use_quantization=True,
    output_dir_name="assertion-correction-model-v2",
    num_epochs=3
):
  """
  Run the complete pipeline for fine-tuning a model for assertion correction.

  Args:
    BASE_DRIVE_PATH: Base path where data and models are stored/will be stored.
    base_model_name: Name of the base model to fine-tune.
    use_quantization: Whether to use quantization for memory efficiency during setup.
    output_dir_name: The directory name used during training to save the model.
    num_epochs: The number of epochs used for training.

  Returns:
    True if fine-tuning and loading were successful, False otherwise.
  """
  print("\n=== Starting Assertion Correction Model Fine-tuning Pipeline ===")

  # Check if BASE_DRIVE_PATH is set
  if not BASE_DRIVE_PATH or not os.path.exists(BASE_DRIVE_PATH):
      print(f"Error: BASE_DRIVE_PATH ('{BASE_DRIVE_PATH}') is not set or does not exist.")
      print("Creating directory...")
      os.makedirs(BASE_DRIVE_PATH, exist_ok=True)

  try:
    # 1. Set up the model with LoRA adapters
    print("\nStep 1: Setting up LoRA model...")
    model, tokenizer = setup_lora_model_for_finetuning(
        base_model_name=base_model_name,
        use_quantization=use_quantization
    )

    if model is None or tokenizer is None:
      print("Failed to set up the model with LoRA adapters. Exiting pipeline.")
      return False
    print("LoRA model setup complete.")

    # 2. Prepare the dataset
    print("\nStep 2: Preparing dataset...")
    train_dataset, eval_dataset = prepare_assertion_correction_dataset(
        BASE_DRIVE_PATH=BASE_DRIVE_PATH,
        tokenizer=tokenizer
    )

    if train_dataset is None or eval_dataset is None:
      print("Failed to prepare the dataset for fine-tuning. Exiting pipeline.")
      return False
    print("Dataset preparation complete.")

    # 3. Run the fine-tuning
    print("\nStep 3: Starting training...")
    trainer = train_assertion_correction_model(
        BASE_DRIVE_PATH=BASE_DRIVE_PATH,
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        output_dir_name=output_dir_name,
        num_train_epochs=num_epochs
    )

    if trainer is None:
      print("Training failed. Exiting pipeline.")
      return False
    print("Training complete.")

    # 4. Test loading the fine-tuned model
    print("\nStep 4: Loading and testing fine-tuned model...")
    test_pipeline = load_finetuned_assertion_model(
        BASE_DRIVE_PATH=BASE_DRIVE_PATH,
        model_dir_name=output_dir_name
    )

    if test_pipeline is None:
      print("Failed to load the fine-tuned model for testing. Exiting pipeline.")
      return False
    print("Fine-tuned model loaded successfully for testing.")

    print("\n=== Assertion Correction Model Fine-tuning Pipeline Completed Successfully! ===")
    return True

  except Exception as e:
    print(f"An error occurred in the fine-tuning pipeline: {e}")
    import traceback
    traceback.print_exc()
    return False


In [ ]:
# Run the Pipeline

# For local execution or if not using Google Drive:
BASE_DRIVE_PATH = "/content/assertion-correction-data"
os.makedirs(BASE_DRIVE_PATH, exist_ok=True)

# Optional: Choose a different model
# base_model_name = "phi-2" # Smaller model option
base_model_name = "deepseek-ai/deepseek-coder-1.3b-instruct" # Default

# Run the complete pipeline
success = run_assertion_model_finetuning_pipeline(
    BASE_DRIVE_PATH=BASE_DRIVE_PATH,
    base_model_name=base_model_name,
    use_quantization=True,  # Set to False if you have enough GPU memory
    output_dir_name="assertion-correction-model-v2",
    num_epochs=3  # Reduce for faster training, increase for better results
)

if success:
    print("Pipeline ran successfully!")
else:
    print("Pipeline encountered errors.")



=== Starting Assertion Correction Model Fine-tuning Pipeline ===

Step 1: Setting up LoRA model...

--- Setting up deepseek-ai/deepseek-coder-1.3b-instruct with LoRA adapters ---
Using 4-bit quantization for memory-efficient training
Configuring LoRA with r=8, alpha=32, dropout=0.05
Trainable parameters: 3145728 (0.42% of total)
trainable params: 3,145,728 || all params: 1,349,617,664 || trainable%: 0.2331
LoRA model setup complete.

Step 2: Preparing dataset...

--- Preparing Assertion Correction Dataset for Fine-tuning ---
Correction summary not found at /content/assertion-correction-data/assertion_correction_results/advanced_correction_summary.csv
Creating sample dataset for demonstration...
Created sample dataset with 20 entries at /content/assertion-correction-data/assertion_correction_results/advanced_correction_summary.csv
Loaded 20 correction examples
Using 20 examples for training
Created 16 training examples and 4 validation examples


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset preparation complete
Dataset preparation complete.

Step 3: Starting training...

--- Starting Fine-tuning for Assertion Correction ---


An error occurred in the fine-tuning pipeline: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'
Pipeline encountered errors.


Traceback (most recent call last):
  File "<ipython-input-41-3d2e1fda3cbd>", line 58, in run_assertion_model_finetuning_pipeline
    trainer = train_assertion_correction_model(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-39-b2cc515f5d50>", line 78, in train_assertion_correction_model
    training_args = TrainingArguments(
                    ^^^^^^^^^^^^^^^^^^
TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'


In [ ]:
# Using the Fine-tuned Model for Assertion Correction

def run_assertion_correction(
    function_code,
    original_test_code,
    model_pipeline=None,
    base_model_name="deepseek-ai/deepseek-coder-1.3b-instruct"
):
    """
    Run assertion correction on a given function and test code.

    Args:
        function_code: The function code to test
        original_test_code: The original test code with weak assertions
        model_pipeline: Optional pre-loaded model pipeline, if None will use base model
        base_model_name: Base model to use if no pipeline is provided

    Returns:
        corrected_test_code: The improved test code with better assertions
    """
    print("\n--- Running Assertion Correction ---")

    # If no fine-tuned model is provided, use the base model directly
    if model_pipeline is None:
        print(f"Using base model: {base_model_name}")
        model_pipeline = pipeline(
            "text-generation",
            model=base_model_name,
            device_map="auto"
        )

    # Format the prompt
    prompt = f"""Function to test:
```python
{function_code}
```

Original test code with weak assertions:
```python
{original_test_code}
```

Improve the assertions in this test code to be more comprehensive and accurate."""

    # Generate improved test code
    print("Generating improved assertions...")

    # Handle different model types appropriately
    if "deepseek" in base_model_name.lower():
        # DeepSeek models use this format
        formatted_prompt = f"<s>[INST] {prompt} [/INST]"
    elif "phi" in base_model_name.lower():
        # Phi models use this format
        formatted_prompt = f"<|user|>\n{prompt}\n<|assistant|>"
    else:
        # Default format
        formatted_prompt = prompt

    # Generate the improved test code
    try:
        response = model_pipeline(
            formatted_prompt,
            max_length=2048,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
        )

        # Extract the generated text
        generated_text = response[0]['generated_text']

        # Extract the improved test code from the response
        # This depends on the model's output format
        if "[/INST]" in generated_text:
            # For DeepSeek format
            improved_code = generated_text.split("[/INST]")[1].strip()
        elif "<|assistant|>" in generated_text:
            # For Phi format
            improved_code = generated_text.split("<|assistant|>")[1].strip()
        else:
            # Default extraction - just take everything after the prompt
            improved_code = generated_text[len(formatted_prompt):].strip()

        # Clean up the code - remove markdown code blocks if present
        if "```python" in improved_code:
            improved_code = improved_code.split("```python")[1]
            if "```" in improved_code:
                improved_code = improved_code.split("```")[0]

        print("Assertion correction complete!")
        return improved_code.strip()

    except Exception as e:
        print(f"Error during assertion correction: {e}")
        return original_test_code

# Example usage:
function_code = """
def add_numbers(a, b):
    return a + b
"""

original_test_code = """
def test_add_numbers():
    assert add_numbers(1, 2) == 3
"""

# use with fine-tuned model:
from transformers import pipeline
BASE_DRIVE_PATH = "/content/assertion-correction-data"
model_pipeline = load_finetuned_assertion_model(BASE_DRIVE_PATH)
improved_test = run_assertion_correction(function_code, original_test_code, model_pipeline)

# Or use with base model:
improved_test = run_assertion_correction(function_code, original_test_code)
print("\nOriginal test code:")
print(original_test_code)
print("\nImproved test code:")
print(improved_test)


In [ ]:
# Model Performance Evaluation

def evaluate_assertion_correction_model(
    BASE_DRIVE_PATH,
    model_pipeline=None,
    model_dir_name="assertion-correction-model-v2",
    test_size=10,
    create_test_data=True
):
    """
    Evaluate the performance of the assertion correction model.

    Args:
        BASE_DRIVE_PATH: Base directory where models are stored
        model_pipeline: Optional pre-loaded model pipeline
        model_dir_name: Name of the model directory
        test_size: Number of test examples to evaluate
        create_test_data: Whether to create synthetic test data

    Returns:
        metrics: Dictionary of performance metrics
    """
    import pandas as pd
    import numpy as np
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    import re
    import ast
    import time

    print("\n=== Evaluating Assertion Correction Model Performance ===")

    # Load the model if not provided
    if model_pipeline is None:
        print(f"Loading model from {model_dir_name}...")
        model_pipeline = load_finetuned_assertion_model(
            BASE_DRIVE_PATH=BASE_DRIVE_PATH,
            model_dir_name=model_dir_name
        )

        if model_pipeline is None:
            print("Failed to load model. Using base model as fallback...")
            model_pipeline = pipeline(
                "text-generation",
                model="deepseek-ai/deepseek-coder-1.3b-instruct",
                device_map="auto"
            )

    # Load or create test data
    test_data_path = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results", "test_data.csv")

    if os.path.exists(test_data_path) and not create_test_data:
        print(f"Loading test data from {test_data_path}")
        test_df = pd.read_csv(test_data_path)
        if len(test_df) > test_size:
            test_df = test_df.sample(test_size, random_state=42)
    else:
        print("Creating synthetic test data...")
        test_data = []

        # Create a variety of test functions with different complexity
        functions = [
            {
                "name": "add_numbers",
                "code": "def add_numbers(a, b):\n    return a + b",
                "weak_test": "def test_add_numbers():\n    assert add_numbers(1, 2) == 3",
                "strong_test": "def test_add_numbers():\n    assert add_numbers(1, 2) == 3\n    assert add_numbers(0, 0) == 0\n    assert add_numbers(-1, 1) == 0\n    assert add_numbers(100, -100) == 0"
            },
            {
                "name": "is_prime",
                "code": "def is_prime(n):\n    if n <= 1:\n        return False\n    if n <= 3:\n        return True\n    if n % 2 == 0 or n % 3 == 0:\n        return False\n    i = 5\n    while i * i <= n:\n        if n % i == 0 or n % (i + 2) == 0:\n            return False\n        i += 6\n    return True",
                "weak_test": "def test_is_prime():\n    assert is_prime(7) == True",
                "strong_test": "def test_is_prime():\n    assert is_prime(7) == True\n    assert is_prime(1) == False\n    assert is_prime(2) == True\n    assert is_prime(4) == False\n    assert is_prime(15) == False\n    assert is_prime(23) == True"
            },
            {
                "name": "reverse_string",
                "code": "def reverse_string(s):\n    return s[::-1]",
                "weak_test": "def test_reverse_string():\n    assert reverse_string('hello') == 'olleh'",
                "strong_test": "def test_reverse_string():\n    assert reverse_string('hello') == 'olleh'\n    assert reverse_string('') == ''\n    assert reverse_string('a') == 'a'\n    assert reverse_string('12345') == '54321'"
            },
            {
                "name": "factorial",
                "code": "def factorial(n):\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial(n-1)",
                "weak_test": "def test_factorial():\n    assert factorial(5) == 120",
                "strong_test": "def test_factorial():\n    assert factorial(5) == 120\n    assert factorial(0) == 1\n    assert factorial(1) == 1\n    assert factorial(3) == 6\n    assert factorial(10) == 3628800"
            },
            {
                "name": "find_max",
                "code": "def find_max(lst):\n    if not lst:\n        return None\n    max_val = lst[0]\n    for item in lst:\n        if item > max_val:\n            max_val = item\n    return max_val",
                "weak_test": "def test_find_max():\n    assert find_max([1, 2, 3, 4, 5]) == 5",
                "strong_test": "def test_find_max():\n    assert find_max([1, 2, 3, 4, 5]) == 5\n    assert find_max([5, 4, 3, 2, 1]) == 5\n    assert find_max([-1, -2, -3]) == -1\n    assert find_max([0]) == 0\n    assert find_max([]) == None"
            }
        ]

        # Generate more examples if needed
        while len(functions) < test_size:
            idx = len(functions) % 5
            base_func = functions[idx]
            new_func = {
                "name": f"{base_func['name']}_{len(functions)}",
                "code": base_func['code'].replace(base_func['name'], f"{base_func['name']}_{len(functions)}"),
                "weak_test": base_func['weak_test'].replace(base_func['name'], f"{base_func['name']}_{len(functions)}"),
                "strong_test": base_func['strong_test'].replace(base_func['name'], f"{base_func['name']}_{len(functions)}")
            }
            functions.append(new_func)

        # Create test data
        for func in functions[:test_size]:
            test_data.append({
                "function_code": func["code"],
                "original_test_code": func["weak_test"],
                "reference_test_code": func["strong_test"]
            })

        test_df = pd.DataFrame(test_data)
        test_df.to_csv(test_data_path, index=False)
        print(f"Created {len(test_df)} test examples and saved to {test_data_path}")

    # Function to count assertions in test code
    def count_assertions(test_code):
        try:
            # Count occurrences of "assert" keyword
            return test_code.count("assert ")
        except:
            return 0

    # Function to check if test code is syntactically valid
    def is_syntactically_valid(test_code):
        try:
            ast.parse(test_code)
            return True
        except SyntaxError:
            return False

    # Evaluate model on test data
    results = []
    start_time = time.time()

    for idx, row in test_df.iterrows():
        print(f"\nEvaluating example {idx+1}/{len(test_df)}...")

        # Get function and test code
        function_code = row["function_code"]
        original_test_code = row["original_test_code"]
        reference_test_code = row["reference_test_code"]

        # Count original assertions
        original_assertions = count_assertions(original_test_code)

        # Count reference assertions
        reference_assertions = count_assertions(reference_test_code)

        # Generate improved test code
        try:
            improved_test_code = run_assertion_correction(
                function_code=function_code,
                original_test_code=original_test_code,
                model_pipeline=model_pipeline
            )

            # Count improved assertions
            improved_assertions = count_assertions(improved_test_code)

            # Check syntactic validity
            improved_syntactic_validity = is_syntactically_valid(improved_test_code)

            # Store results
            results.append({
                "original_assertions": original_assertions,
                "improved_assertions": improved_assertions,
                "reference_assertions": reference_assertions,
                "improved_syntactic_validity": improved_syntactic_validity,
                "assertion_increase": improved_assertions > original_assertions,
                "assertion_count_match": improved_assertions >= reference_assertions,
                "original_test_code": original_test_code,
                "improved_test_code": improved_test_code,
                "reference_test_code": reference_test_code
            })

        except Exception as e:
            print(f"Error evaluating example {idx+1}: {e}")
            results.append({
                "original_assertions": original_assertions,
                "improved_assertions": 0,
                "reference_assertions": reference_assertions,
                "improved_syntactic_validity": False,
                "assertion_increase": False,
                "assertion_count_match": False,
                "original_test_code": original_test_code,
                "improved_test_code": "ERROR",
                "reference_test_code": reference_test_code
            })

    # Calculate evaluation time
    eval_time = time.time() - start_time

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Calculate metrics
    metrics = {
        "total_examples": len(results_df),
        "syntactically_valid": results_df["improved_syntactic_validity"].sum(),
        "syntactic_validity_rate": results_df["improved_syntactic_validity"].mean() * 100,
        "assertion_increase_count": results_df["assertion_increase"].sum(),
        "assertion_increase_rate": results_df["assertion_increase"].mean() * 100,
        "assertion_count_match": results_df["assertion_count_match"].sum(),
        "assertion_count_match_rate": results_df["assertion_count_match"].mean() * 100,
        "avg_original_assertions": results_df["original_assertions"].mean(),
        "avg_improved_assertions": results_df["improved_assertions"].mean(),
        "avg_reference_assertions": results_df["reference_assertions"].mean(),
        "assertion_improvement_ratio": results_df["improved_assertions"].sum() / results_df["original_assertions"].sum(),
        "evaluation_time_seconds": eval_time,
        "evaluation_time_per_example": eval_time / len(results_df)
    }

    # Save results
    results_path = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results", "model_evaluation_results.csv")
    results_df.to_csv(results_path, index=False)

    metrics_path = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results", "model_evaluation_metrics.csv")
    pd.DataFrame([metrics]).to_csv(metrics_path, index=False)

    # Print metrics
    print("\n=== Model Performance Metrics ===")
    print(f"Total examples evaluated: {metrics['total_examples']}")
    print(f"Syntactic validity rate: {metrics['syntactic_validity_rate']:.2f}%")
    print(f"Assertion increase rate: {metrics['assertion_increase_rate']:.2f}%")
    print(f"Assertion count match rate: {metrics['assertion_count_match_rate']:.2f}%")
    print(f"Average assertions - Original: {metrics['avg_original_assertions']:.2f}, Improved: {metrics['avg_improved_assertions']:.2f}, Reference: {metrics['avg_reference_assertions']:.2f}")
    print(f"Assertion improvement ratio: {metrics['assertion_improvement_ratio']:.2f}x")
    print(f"Evaluation time: {metrics['evaluation_time_seconds']:.2f} seconds ({metrics['evaluation_time_per_example']:.2f} seconds per example)")
    print(f"Results saved to {results_path}")
    print(f"Metrics saved to {metrics_path}")

    return metrics, results_df

# Example usage:
# run evaluation
BASE_DRIVE_PATH = "/content/assertion-correction-data"
metrics, results = evaluate_assertion_correction_model(BASE_DRIVE_PATH, test_size=5)

# Create a visual performance report
def create_performance_visualization(BASE_DRIVE_PATH):
    """
    Create visualizations of model performance metrics.

    Args:
        BASE_DRIVE_PATH: Base directory where evaluation results are stored
    """
    import matplotlib.pyplot as plt
    import pandas as pd
    import os

    metrics_path = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results", "model_evaluation_metrics.csv")
    results_path = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results", "model_evaluation_results.csv")

    if not os.path.exists(metrics_path) or not os.path.exists(results_path):
        print("Evaluation results not found. Please run evaluation first.")
        return

    # Load metrics and results
    metrics_df = pd.read_csv(metrics_path)
    results_df = pd.read_csv(results_path)

    # Create output directory
    viz_dir = os.path.join(BASE_DRIVE_PATH, "assertion_correction_results", "visualizations")
    os.makedirs(viz_dir, exist_ok=True)

    # 1. Success rates bar chart
    plt.figure(figsize=(10, 6))
    success_rates = [
        metrics_df["syntactic_validity_rate"].values[0],
        metrics_df["assertion_increase_rate"].values[0],
        metrics_df["assertion_count_match_rate"].values[0]
    ]
    labels = ["Syntactic Validity", "Assertion Increase", "Assertion Count Match"]
    plt.bar(labels, success_rates, color=["#4CAF50", "#2196F3", "#FFC107"])
    plt.ylabel("Success Rate (%)")
    plt.title("Model Performance Success Rates")
    plt.ylim(0, 100)
    for i, v in enumerate(success_rates):
        plt.text(i, v + 2, f"{v:.1f}%", ha='center')
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, "success_rates.png"))

    # 2. Assertion counts comparison
    plt.figure(figsize=(10, 6))
    assertion_counts = [
        metrics_df["avg_original_assertions"].values[0],
        metrics_df["avg_improved_assertions"].values[0],
        metrics_df["avg_reference_assertions"].values[0]
    ]
    labels = ["Original", "Model-Improved", "Reference"]
    plt.bar(labels, assertion_counts, color=["#E57373", "#64B5F6", "#81C784"])
    plt.ylabel("Average Number of Assertions")
    plt.title("Assertion Count Comparison")
    for i, v in enumerate(assertion_counts):
        plt.text(i, v + 0.1, f"{v:.2f}", ha='center')
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, "assertion_counts.png"))

    # 3. Assertion improvement distribution
    plt.figure(figsize=(12, 6))
    improvement = results_df["improved_assertions"] - results_df["original_assertions"]
    plt.hist(improvement, bins=range(min(improvement)-1, max(improvement)+2), alpha=0.7, color="#7986CB")
    plt.xlabel("Assertion Count Improvement")
    plt.ylabel("Number of Examples")
    plt.title("Distribution of Assertion Count Improvement")
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, "improvement_distribution.png"))

    # 4. Scatter plot of original vs improved assertions
    plt.figure(figsize=(10, 6))
    plt.scatter(results_df["original_assertions"], results_df["improved_assertions"],
                alpha=0.7, s=100, c="#FF7043")
    plt.xlabel("Original Assertion Count")
    plt.ylabel("Improved Assertion Count")
    plt.title("Original vs. Improved Assertion Counts")

    # Add diagonal line (y=x)
    max_val = max(results_df["original_assertions"].max(), results_df["improved_assertions"].max()) + 1
    plt.plot([0, max_val], [0, max_val], 'k--', alpha=0.5)

    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, "original_vs_improved.png"))

    # 5. Performance summary table
    plt.figure(figsize=(12, 4))
    plt.axis('off')
    table_data = [
        ["Metric", "Value"],
        ["Total Examples", f"{metrics_df['total_examples'].values[0]}"],
        ["Syntactic Validity", f"{metrics_df['syntactic_validity_rate'].values[0]:.2f}%"],
        ["Assertion Increase Rate", f"{metrics_df['assertion_increase_rate'].values[0]:.2f}%"],
        ["Assertion Count Match Rate", f"{metrics_df['assertion_count_match_rate'].values[0]:.2f}%"],
        ["Assertion Improvement Ratio", f"{metrics_df['assertion_improvement_ratio'].values[0]:.2f}x"],
        ["Evaluation Time", f"{metrics_df['evaluation_time_seconds'].values[0]:.2f} seconds"]
    ]

    table = plt.table(cellText=table_data, loc='center', cellLoc='center', colWidths=[0.4, 0.4])
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1, 2)
    plt.title("Performance Summary", pad=20, fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, "performance_summary.png"))

    print(f"Visualizations saved to {viz_dir}")
    return viz_dir

# Example usage:
# create visualizations after running evaluation
BASE_DRIVE_PATH = "/content/assertion-correction-data"
viz_dir = create_performance_visualization(BASE_DRIVE_PATH)


------------------------end-------------------------